In [13]:
from csgo.parser import DemoParser
from csgo.analytics import stats
import numpy
import pandas
import os

In [2]:
def getDemoPaths(directory):
    demoPaths = []
    for filename in os.listdir(directory):
        demoPaths.append(os.path.join(directory, filename))
    return demoPaths


In [3]:
def generateDemoParsers(paths):
    demo_parsers = []
    for x in paths:
        demoParser = DemoParser(demofile=x, demo_id = x.split('/')[-1].split('.')[0].split('_')[0], parse_rate=128)
        demo_parsers.append(demoParser)
    print("# Demo Parsers: " + str(len(demo_parsers)))
    return demo_parsers

In [4]:
def generateData(parsers):
    killData, roundData, damageData, frameData, playerFrames, bombEvents, grenadeData, flashData = [], [], [], [], [], [], [], []
    for x in parsers:
        x.parse()
        killData.append(x._parse_kills(return_type="df"))
        roundData.append(x._parse_rounds(return_type="df"))
        damageData.append(x._parse_damages(return_type="df"))
        frameData.append(x._parse_frames(return_type="df"))
        playerFrames.append(x._parse_player_frames(return_type="df"))
        bombEvents.append(x._parse_bomb_events(return_type="df"))
        grenadeData.append(x._parse_grenades(return_type="df"))
        flashData.append(x._parse_flashes(return_type="df"))
    return ( killData, roundData, damageData, frameData, playerFrames, bombEvents, grenadeData, flashData)

In [5]:
def createVsMatrix():
    return {'5' : {'5' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '4' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '3' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '2' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '1' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}},
                      '4' : {'5' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '4' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '3' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '2' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '1' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}}, 
                      '3' : {'5' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '4' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '3' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '2' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '1' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}},
                      '2' : {'5' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '4' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '3' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '2' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '1' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}},
                      '1' : {'5' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '4' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '3' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '2' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}, '1' : {'Win' : 0, 'Lose' : 0, 'Total' : 0, 'changed' : False}}}

In [6]:
def createCT_T_VS_Breakdowns(totalData, ct_breakdown, t_breakdown, team):
    for x in totalData:
        for y in x['GameRounds']:
            winningTeam = (y['WinningTeam'] == team)
            for z in y['Frames']:
                if(z['T']['TeamName'] == team):
                    if(z['T']['AlivePlayers'] != 0 and z['CT']['AlivePlayers'] != 0):
                        if(not t_breakdown[str(z['T']['AlivePlayers'])][str(z['CT']['AlivePlayers'])]['changed']):
                            t_breakdown[str(z['T']['AlivePlayers'])][str(z['CT']['AlivePlayers'])]['Total'] += 1
                            if(winningTeam):
                                t_breakdown[str(z['T']['AlivePlayers'])][str(z['CT']['AlivePlayers'])]['Win'] += 1
                            else:
                                t_breakdown[str(z['T']['AlivePlayers'])][str(z['CT']['AlivePlayers'])]['Lose'] += 1
                            t_breakdown[str(z['T']['AlivePlayers'])][str(z['CT']['AlivePlayers'])]['changed'] = True
                elif(z['CT']['TeamName'] == team):
                    if(z['T']['AlivePlayers'] != 0 and z['CT']['AlivePlayers'] != 0):
                        if(not ct_breakdown[str(z['CT']['AlivePlayers'])][str(z['T']['AlivePlayers'])]['changed']):
                            ct_breakdown[str(z['CT']['AlivePlayers'])][str(z['T']['AlivePlayers'])]['Total'] += 1
                            if(winningTeam):
                                ct_breakdown[str(z['CT']['AlivePlayers'])][str(z['T']['AlivePlayers'])]['Win'] += 1
                            else:
                                ct_breakdown[str(z['CT']['AlivePlayers'])][str(z['T']['AlivePlayers'])]['Lose'] += 1
                            ct_breakdown[str(z['CT']['AlivePlayers'])][str(z['T']['AlivePlayers'])]['changed'] = True
            for i in ct_breakdown:
                for j in ct_breakdown[i]:
                    ct_breakdown[i][j]['changed'] = False
            for i in t_breakdown:
                for j in t_breakdown[i]:
                    t_breakdown[i][j]['changed'] = False

In [7]:
def generateWr(side_breakdown):
    side_wrs = {}
    for x in side_breakdown:
        for y in side_breakdown[x]:
            side_wrs[str(x) + " vs " + str(y)] = side_breakdown[x][y]['Win'] / side_breakdown[x][y]['Total']
    return side_wrs

In [8]:
def divideDataByMaps(data):
    dataByMap = {}
    for x in data.MapName.unique():
        dataByMap[x] = data[data['MapName'] == x]
    return dataByMap

In [9]:
def concat_data(aggregate_data):
    final_df = None
    for x in aggregate_data:
        final_df = pandas.concat([final_df, x], ignore_index=True)
    return final_df

In [89]:
faze_path = "/mnt/d/TOP_10/Faze"
g2_path = "/mnt/d/TOP_10/G2"
heroic_path = "/mnt/d/TOP_10/Heroic"
nip_path = "/mnt/d/TOP_10/NiP"
vp_path = "/mnt/d/TOP_10/VP"
astralis_path = "/mnt/d/TOP_10/Astralis"
og_path = "/mnt/d/TOP_10/OG"
big_path = "/mnt/d/TOP_10/Big"
spirit_path = "/mnt/d/TOP_10/Spirit"
forze_path = "/mnt/d/TOP_10/Forze"
complexity_path = "/mnt/d/TOP_10/Complexity"
ence_path = "/mnt/d/TOP_10/Ence"
mous_path = "/mnt/d/TOP_10/Mous"

In [24]:
fKillData, fRoundData, fDamageData, fFrameData, fPlayerFrames, fBombEvents, fGrenadeData, fFlashData = generateData(generateDemoParsers(getDemoPaths(faze_path)))
#bkillData, broundData, bdamageData, bframeData, bplayerFrames, bbombEvents, bgrenadeData, bflashData = generateData(generateDemoParsers(getDemoPaths(group_b_demo_paths)))


12:52:06 [INFO] Go version>=1.14.0
12:52:06 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Faze/big-vs-faze-m1-dust2.dem
12:52:06 [INFO] Setting demo id to big-vs-faze-m1-dust2
12:52:06 [INFO] Setting parse rate to 128
12:52:06 [INFO] Go version>=1.14.0
12:52:06 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Faze/big-vs-faze-m2-inferno.dem
12:52:06 [INFO] Setting demo id to big-vs-faze-m2-inferno
12:52:06 [INFO] Setting parse rate to 128
12:52:07 [INFO] Go version>=1.14.0
12:52:07 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Faze/complexity-vs-faze-m1-dust2.dem
12:52:07 [INFO] Setting demo id to complexity-vs-faze-m1-dust2
12:52:07 [INFO] Setting parse rate to 128
12:52:07 [INFO] Go version>=1.14.0
12:52:07 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Faze/complexity-vs-faze-m2-nuke.dem
12:52:07 [INFO] Setting demo id to complexity-vs-faze-m2-nuke
12:52:07 [INFO] Setting parse rate to 128
12:52:07 [INFO] Go version>=1.14.0

# Demo Parsers: 27


12:52:29 [INFO] Wrote demo parse output to big-vs-faze-m1-dust2.json
12:52:29 [INFO] Reading in JSON from big-vs-faze-m1-dust2.json
12:52:29 [INFO] JSON data loaded, available in the `json` attribute to parser
12:52:29 [INFO] Successfully parsed JSON output
12:52:29 [INFO] Successfully returned JSON output
12:52:29 [INFO] Parsed kills to Pandas DataFrame
12:52:29 [INFO] Parsed rounds to Pandas DataFrame
12:52:29 [INFO] Parsed damages to Pandas DataFrame
12:52:29 [INFO] Parsed frames to Pandas DataFrame
12:52:29 [INFO] Parsed player frames to Pandas DataFrame
12:52:29 [INFO] Parsed bomb_events to Pandas DataFrame
12:52:29 [INFO] Parsed grenades to Pandas DataFrame
12:52:29 [INFO] Parsed flashes to Pandas DataFrame
12:52:29 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
12:52:29 [INFO] Looking for file at /mnt/d/TOP_10/Faze/big-vs-faze-m2-inferno.dem
12:52:43 [INFO] Wrote demo parse output to big-vs-faze-m2-inferno.json
12:52:43 [INFO] Reading in JSON from big-

12:55:01 [INFO] Reading in JSON from faze-vs-evil-geniuses-m1-inferno.json
12:55:02 [INFO] JSON data loaded, available in the `json` attribute to parser
12:55:02 [INFO] Successfully parsed JSON output
12:55:02 [INFO] Successfully returned JSON output
12:55:02 [INFO] Parsed kills to Pandas DataFrame
12:55:02 [INFO] Parsed rounds to Pandas DataFrame
12:55:02 [INFO] Parsed damages to Pandas DataFrame
12:55:02 [INFO] Parsed frames to Pandas DataFrame
12:55:02 [INFO] Parsed player frames to Pandas DataFrame
12:55:02 [INFO] Parsed bomb_events to Pandas DataFrame
12:55:02 [INFO] Parsed grenades to Pandas DataFrame
12:55:02 [INFO] Parsed flashes to Pandas DataFrame
12:55:02 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
12:55:02 [INFO] Looking for file at /mnt/d/TOP_10/Faze/faze-vs-evil-geniuses-m2-mirage.dem
12:55:15 [INFO] Wrote demo parse output to faze-vs-evil-geniuses-m2-mirage.json
12:55:15 [INFO] Reading in JSON from faze-vs-evil-geniuses-m2-mirage.json
12:55:

12:57:47 [INFO] Reading in JSON from natus-vincere-vs-faze-m1-inferno.json
12:57:48 [INFO] JSON data loaded, available in the `json` attribute to parser
12:57:48 [INFO] Successfully parsed JSON output
12:57:48 [INFO] Successfully returned JSON output
12:57:48 [INFO] Parsed kills to Pandas DataFrame
12:57:48 [INFO] Parsed rounds to Pandas DataFrame
12:57:48 [INFO] Parsed damages to Pandas DataFrame
12:57:48 [INFO] Parsed frames to Pandas DataFrame
12:57:48 [INFO] Parsed player frames to Pandas DataFrame
12:57:48 [INFO] Parsed bomb_events to Pandas DataFrame
12:57:48 [INFO] Parsed grenades to Pandas DataFrame
12:57:48 [INFO] Parsed flashes to Pandas DataFrame
12:57:48 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
12:57:48 [INFO] Looking for file at /mnt/d/TOP_10/Faze/natus-vincere-vs-faze-m2-mirage.dem
12:58:03 [INFO] Wrote demo parse output to natus-vincere-vs-faze-m2-mirage.json
12:58:03 [INFO] Reading in JSON from natus-vincere-vs-faze-m2-mirage.json
12:58:

In [25]:
finalfKillData = concat_data(fKillData)
finalfRoundData = concat_data(fRoundData)
finalfDamageData = concat_data(fDamageData)
finalfGrenadeData = concat_data(fGrenadeData)
finalfFlashData = concat_data(fFlashData)
finalfBombData = concat_data(fBombEvents)

In [30]:
fkillMapData = divideDataByMaps(finalfKillData)
froundMapData = divideDataByMaps(finalfRoundData)
fdamageMapData = divideDataByMaps(finalfDamageData)
fgrenadeMapData = divideDataByMaps(finalfGrenadeData)
fflashMapData = divideDataByMaps(finalfFlashData)
fBombMapData = divideDataByMaps(finalfBombData)

In [31]:
f_kill_stats = {}
for x in fkillMapData.keys():
    f_kill_stats[x] = stats.kill_stats(fkillMapData[x], froundMapData[x])

In [32]:
f_kill_stats['de_mirage']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Twistzz,103,94,11.0,9,75,15.0,1.095745,0.715278,0.728155
1,karrigan,66,104,9.0,-38,32,7.0,0.634615,0.458333,0.484848
2,broky,61,60,5.0,1,21,5.0,1.016667,0.423611,0.344262
3,rain,53,69,7.0,-16,34,13.0,0.768116,0.368056,0.641509
4,olofmeister,53,65,13.0,-12,29,6.0,0.815385,0.368056,0.547170
5,broky,46,29,4.0,17,15,7.0,1.586207,0.319444,0.326087
6,rain,36,29,3.0,7,19,5.0,1.241379,0.250000,0.527778
7,olofmeister,28,29,4.0,-1,14,3.0,0.965517,0.194444,0.500000
8,hallzerk * VIE,24,21,3.0,3,9,6.0,1.142857,0.166667,0.375000
9,electronic,23,20,2.0,3,10,2.0,1.150000,0.159722,0.434783


In [33]:
f_kill_stats['de_overpass']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,karrigan,68,57,11.0,11,34,11.0,1.192982,0.839506,0.500000
1,broky,64,51,11.0,13,24,7.0,1.254902,0.790123,0.375000
2,rain,62,55,9.0,7,33,12.0,1.127273,0.765432,0.532258
3,Twistzz,51,45,7.0,6,28,5.0,1.133333,0.629630,0.549020
4,olofmeister,46,51,4.0,-5,24,4.0,0.901961,0.567901,0.521739
5,ropz,30,20,3.0,10,15,3.0,1.500000,0.370370,0.500000
6,Jackinho,25,21,3.0,4,2,6.0,1.190476,0.308642,0.080000
7,mezii,25,20,8.0,5,12,2.0,1.250000,0.308642,0.480000
8,KRIMZ,20,17,2.0,3,9,1.0,1.176471,0.246914,0.450000
9,Brollan,20,22,8.0,-2,6,6.0,0.909091,0.246914,0.300000


In [34]:
f_kill_stats['de_inferno']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,karrigan,100,104,18.0,-4,51,19.0,0.961538,0.649351,0.510000
1,Twistzz,97,92,21.0,5,62,15.0,1.054348,0.629870,0.639175
2,olofmeister,83,62,9.0,21,46,10.0,1.338710,0.538961,0.554217
3,broky,72,70,12.0,2,30,9.0,1.028571,0.467532,0.416667
4,rain,69,86,21.0,-17,44,18.0,0.802326,0.448052,0.637681
5,mantuu,29,16,3.0,13,7,5.0,1.812500,0.188312,0.241379
6,Brehze,27,18,1.0,9,13,4.0,1.500000,0.175325,0.481481
7,rain,24,20,6.0,4,15,4.0,1.200000,0.155844,0.625000
8,flameZ,22,18,4.0,4,10,5.0,1.222222,0.142857,0.454545
9,Jame,21,12,2.0,9,6,2.0,1.750000,0.136364,0.285714


In [35]:
f_kill_stats['de_ancient']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Twistzz,51,41,9.0,10,33,4.0,1.243902,0.739130,0.647059
1,karrigan,41,46,5.0,-5,20,10.0,0.891304,0.594203,0.487805
2,broky,37,22,3.0,15,7,5.0,1.681818,0.536232,0.189189
3,rain,31,26,9.0,5,12,5.0,1.192308,0.449275,0.387097
4,olofmeister,31,19,4.0,12,17,5.0,1.631579,0.449275,0.548387
5,mantuu,25,13,0.0,12,4,7.0,1.923077,0.362319,0.160000
6,frozen,24,14,2.0,10,10,7.0,1.714286,0.347826,0.416667
7,flameZ,24,22,1.0,2,15,4.0,1.090909,0.347826,0.625000
8,ropz,21,12,2.0,9,13,4.0,1.750000,0.304348,0.619048
9,dexter,20,12,4.0,8,12,3.0,1.666667,0.289855,0.600000


In [36]:
f_kill_stats['de_vertigo']

KeyError: 'de_vertigo'

In [37]:
f_kill_stats['de_nuke']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Twistzz,67,78,11.0,-11,41,7.0,0.858974,0.598214,0.611940
1,karrigan,57,84,19.0,-27,31,9.0,0.678571,0.508929,0.543860
2,rain,49,50,7.0,-1,29,10.0,0.980000,0.437500,0.591837
3,broky,44,41,4.0,3,9,3.0,1.073171,0.392857,0.204545
4,poizon,41,30,7.0,11,20,8.0,1.366667,0.366071,0.487805
5,es3tag,38,25,7.0,13,11,8.0,1.520000,0.339286,0.289474
6,olofmeister,36,41,5.0,-5,16,1.0,0.878049,0.321429,0.444444
7,Perfecto,34,20,5.0,14,17,0.0,1.700000,0.303571,0.500000
8,jks,33,22,5.0,11,16,3.0,1.500000,0.294643,0.484848
9,rain,31,39,6.0,-8,19,8.0,0.794872,0.276786,0.612903


In [38]:
f_kill_stats['de_dust2']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Twistzz,93,80,9.0,13,62,10.0,1.162500,0.688889,0.666667
1,karrigan,75,86,20.0,-11,37,11.0,0.872093,0.555556,0.493333
2,broky,53,44,10.0,9,14,5.0,1.204545,0.392593,0.264151
3,rain,45,50,9.0,-5,27,13.0,0.900000,0.333333,0.600000
4,broky,35,33,6.0,2,15,6.0,1.060606,0.259259,0.428571
5,olofmeister,34,50,6.0,-16,19,7.0,0.680000,0.251852,0.558824
6,rain,34,42,11.0,-8,26,11.0,0.809524,0.251852,0.764706
7,olofmeister,30,38,7.0,-8,19,2.0,0.789474,0.222222,0.633333
8,Jame,26,9,1.0,17,4,4.0,2.888889,0.192593,0.153846
9,jks,26,18,3.0,8,22,2.0,1.444444,0.192593,0.846154


In [39]:
gKillData, gRoundData, gDamageData, gFrameData, gPlayerFrames, gBombEvents, gGrenadeData, gFlashData = generateData(generateDemoParsers(getDemoPaths(g2_path)))

13:00:33 [INFO] Go version>=1.14.0
13:00:33 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/G2/g2-vs-astralis-m1-nuke_2349928.dem
13:00:33 [INFO] Setting demo id to g2-vs-astralis-m1-nuke
13:00:33 [INFO] Setting parse rate to 128
13:00:33 [INFO] Go version>=1.14.0
13:00:33 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/G2/g2-vs-astralis-m2-dust2_2349928.dem
13:00:33 [INFO] Setting demo id to g2-vs-astralis-m2-dust2
13:00:33 [INFO] Setting parse rate to 128
13:00:33 [INFO] Go version>=1.14.0
13:00:33 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/G2/g2-vs-astralis-m3-inferno_2349928.dem
13:00:33 [INFO] Setting demo id to g2-vs-astralis-m3-inferno
13:00:33 [INFO] Setting parse rate to 128
13:00:34 [INFO] Go version>=1.14.0
13:00:34 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/G2/g2-vs-big-m1-dust2.dem
13:00:34 [INFO] Setting demo id to g2-vs-big-m1-dust2
13:00:34 [INFO] Setting parse rate to 128
13:00:34 [INFO] Go version>=1.14

13:00:43 [INFO] Setting parse rate to 128
13:00:44 [INFO] Go version>=1.14.0
13:00:44 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/G2/g2-vs-virtus-pro-m1-dust2_2350345.dem
13:00:44 [INFO] Setting demo id to g2-vs-virtus-pro-m1-dust2
13:00:44 [INFO] Setting parse rate to 128
13:00:44 [INFO] Go version>=1.14.0
13:00:44 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/G2/g2-vs-virtus-pro-m2-mirage_2350345.dem
13:00:44 [INFO] Setting demo id to g2-vs-virtus-pro-m2-mirage
13:00:44 [INFO] Setting parse rate to 128
13:00:44 [INFO] Go version>=1.14.0
13:00:44 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/G2/g2-vs-virtus-pro-m3-vertigo_2350345.dem
13:00:44 [INFO] Setting demo id to g2-vs-virtus-pro-m3-vertigo
13:00:44 [INFO] Setting parse rate to 128
13:00:44 [INFO] Go version>=1.14.0
13:00:44 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/G2/gambit-vs-g2-m1-mirage_2349923.dem
13:00:44 [INFO] Setting demo id to gambit-vs-g2-m1-mirage


# Demo Parsers: 45


13:01:04 [INFO] Wrote demo parse output to g2-vs-astralis-m1-nuke.json
13:01:04 [INFO] Reading in JSON from g2-vs-astralis-m1-nuke.json
13:01:05 [INFO] JSON data loaded, available in the `json` attribute to parser
13:01:05 [INFO] Successfully parsed JSON output
13:01:05 [INFO] Successfully returned JSON output
13:01:05 [INFO] Parsed kills to Pandas DataFrame
13:01:05 [INFO] Parsed rounds to Pandas DataFrame
13:01:05 [INFO] Parsed damages to Pandas DataFrame
13:01:05 [INFO] Parsed frames to Pandas DataFrame
13:01:05 [INFO] Parsed player frames to Pandas DataFrame
13:01:05 [INFO] Parsed bomb_events to Pandas DataFrame
13:01:05 [INFO] Parsed grenades to Pandas DataFrame
13:01:05 [INFO] Parsed flashes to Pandas DataFrame
13:01:05 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:01:05 [INFO] Looking for file at /mnt/d/TOP_10/G2/g2-vs-astralis-m2-dust2_2349928.dem
13:01:22 [INFO] Wrote demo parse output to g2-vs-astralis-m2-dust2.json
13:01:22 [INFO] Reading in JS

13:03:33 [INFO] Successfully parsed JSON output
13:03:33 [INFO] Successfully returned JSON output
13:03:33 [INFO] Parsed kills to Pandas DataFrame
13:03:33 [INFO] Parsed rounds to Pandas DataFrame
13:03:33 [INFO] Parsed damages to Pandas DataFrame
13:03:33 [INFO] Parsed frames to Pandas DataFrame
13:03:33 [INFO] Parsed player frames to Pandas DataFrame
13:03:33 [INFO] Parsed bomb_events to Pandas DataFrame
13:03:33 [INFO] Parsed grenades to Pandas DataFrame
13:03:33 [INFO] Parsed flashes to Pandas DataFrame
13:03:33 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:03:33 [INFO] Looking for file at /mnt/d/TOP_10/G2/g2-vs-big-m3-vertigo.dem
13:03:47 [INFO] Wrote demo parse output to g2-vs-big-m3-vertigo.json
13:03:47 [INFO] Reading in JSON from g2-vs-big-m3-vertigo.json
13:03:47 [INFO] JSON data loaded, available in the `json` attribute to parser
13:03:47 [INFO] Successfully parsed JSON output
13:03:47 [INFO] Successfully returned JSON output
13:03:47 [INFO] Pa

13:05:49 [INFO] Parsed kills to Pandas DataFrame
13:05:49 [INFO] Parsed rounds to Pandas DataFrame
13:05:49 [INFO] Parsed damages to Pandas DataFrame
13:05:49 [INFO] Parsed frames to Pandas DataFrame
13:05:49 [INFO] Parsed player frames to Pandas DataFrame
13:05:49 [INFO] Parsed bomb_events to Pandas DataFrame
13:05:49 [INFO] Parsed grenades to Pandas DataFrame
13:05:49 [INFO] Parsed flashes to Pandas DataFrame
13:05:49 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:05:49 [INFO] Looking for file at /mnt/d/TOP_10/G2/g2-vs-faze-m2-inferno.dem
13:06:05 [INFO] Wrote demo parse output to g2-vs-faze-m2-inferno.json
13:06:05 [INFO] Reading in JSON from g2-vs-faze-m2-inferno.json
13:06:05 [INFO] JSON data loaded, available in the `json` attribute to parser
13:06:05 [INFO] Successfully parsed JSON output
13:06:05 [INFO] Successfully returned JSON output
13:06:05 [INFO] Parsed kills to Pandas DataFrame
13:06:05 [INFO] Parsed rounds to Pandas DataFrame
13:06:05 [INFO

13:08:26 [INFO] Parsed frames to Pandas DataFrame
13:08:27 [INFO] Parsed player frames to Pandas DataFrame
13:08:27 [INFO] Parsed bomb_events to Pandas DataFrame
13:08:27 [INFO] Parsed grenades to Pandas DataFrame
13:08:27 [INFO] Parsed flashes to Pandas DataFrame
13:08:27 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:08:27 [INFO] Looking for file at /mnt/d/TOP_10/G2/g2-vs-nip-m1-ancient_2351004.dem
13:08:40 [INFO] Wrote demo parse output to g2-vs-nip-m1-ancient.json
13:08:40 [INFO] Reading in JSON from g2-vs-nip-m1-ancient.json
13:08:40 [INFO] JSON data loaded, available in the `json` attribute to parser
13:08:40 [INFO] Successfully parsed JSON output
13:08:40 [INFO] Successfully returned JSON output
13:08:40 [INFO] Parsed kills to Pandas DataFrame
13:08:40 [INFO] Parsed rounds to Pandas DataFrame
13:08:40 [INFO] Parsed damages to Pandas DataFrame
13:08:40 [INFO] Parsed frames to Pandas DataFrame
13:08:40 [INFO] Parsed player frames to Pandas DataFrame
1

13:10:51 [INFO] Parsed bomb_events to Pandas DataFrame
13:10:51 [INFO] Parsed grenades to Pandas DataFrame
13:10:51 [INFO] Parsed flashes to Pandas DataFrame
13:10:51 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:10:51 [INFO] Looking for file at /mnt/d/TOP_10/G2/gambit-vs-g2-m1-mirage_2349923.dem
13:11:08 [INFO] Wrote demo parse output to gambit-vs-g2-m1-mirage.json
13:11:08 [INFO] Reading in JSON from gambit-vs-g2-m1-mirage.json
13:11:08 [INFO] JSON data loaded, available in the `json` attribute to parser
13:11:08 [INFO] Successfully parsed JSON output
13:11:08 [INFO] Successfully returned JSON output
13:11:08 [INFO] Parsed kills to Pandas DataFrame
13:11:08 [INFO] Parsed rounds to Pandas DataFrame
13:11:08 [INFO] Parsed damages to Pandas DataFrame
13:11:08 [INFO] Parsed frames to Pandas DataFrame
13:11:08 [INFO] Parsed player frames to Pandas DataFrame
13:11:08 [INFO] Parsed bomb_events to Pandas DataFrame
13:11:08 [INFO] Parsed grenades to Pandas DataF

In [40]:
finalgKillData = concat_data(gKillData)
finalgRoundData = concat_data(gRoundData)
finalgDamageData = concat_data(gDamageData)
finalgGrenadeData = concat_data(gGrenadeData)
finalgFlashData = concat_data(gFlashData)
finalgBombData = concat_data(gBombEvents)

In [41]:
gkillMapData = divideDataByMaps(finalgKillData)
groundMapData = divideDataByMaps(finalgRoundData)
gdamageMapData = divideDataByMaps(finalgDamageData)
ggrenadeMapData = divideDataByMaps(finalgGrenadeData)
gflashMapData = divideDataByMaps(finalgFlashData)
gBombMapData = divideDataByMaps(finalgBombData)

In [42]:
g_kill_stats = {}
for x in gkillMapData.keys():
    g_kill_stats[x] = stats.kill_stats(gkillMapData[x], groundMapData[x])

In [43]:
g_kill_stats['de_mirage']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,nexa,177,182,24.0,-5,95,16.0,0.972527,0.610345,0.536723
1,AmaNEk,176,175,27.0,1,46,25.0,1.005714,0.606897,0.261364
2,huNter-,164,147,26.0,17,87,16.0,1.115646,0.565517,0.530488
3,JACKZ,160,194,39.0,-34,106,30.0,0.824742,0.551724,0.662500
4,NiKo,132,102,21.0,30,69,32.0,1.294118,0.455172,0.522727
5,NiKo,118,81,15.0,37,63,26.0,1.456790,0.406897,0.533898
6,Perfecto,40,30,3.0,10,20,2.0,1.333333,0.137931,0.500000
7,electronic,40,37,3.0,3,16,7.0,1.081081,0.137931,0.400000
8,b1t,38,34,6.0,4,29,9.0,1.117647,0.131034,0.763158
9,s1mple,38,32,8.0,6,10,8.0,1.187500,0.131034,0.263158


In [44]:
g_kill_stats['de_overpass']

KeyError: 'de_overpass'

In [45]:
g_kill_stats['de_inferno']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,AmaNEk,172,162,26.0,10,81,31.0,1.061728,0.677165,0.470930
1,JACKZ,152,162,22.0,-10,98,19.0,0.938272,0.598425,0.644737
2,nexa,137,147,25.0,-10,79,15.0,0.931973,0.539370,0.576642
3,huNter-,134,112,25.0,22,69,23.0,1.196429,0.527559,0.514925
4,NiKo,116,97,15.0,19,62,34.0,1.195876,0.456693,0.534483
5,NiKo,78,63,13.0,15,42,13.0,1.238095,0.307087,0.538462
6,huNter,59,50,11.0,9,33,6.0,1.180000,0.232283,0.559322
7,REZ,47,40,7.0,7,25,3.0,1.175000,0.185039,0.531915
8,hampus,45,42,9.0,3,19,11.0,1.071429,0.177165,0.422222
9,LNZ,36,34,3.0,2,16,5.0,1.058824,0.141732,0.444444


In [46]:
g_kill_stats['de_ancient']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,NiKo,44,27,12.0,17,22,7.0,1.629630,0.897959,0.500000
1,JACKZ,33,26,4.0,7,19,7.0,1.269231,0.673469,0.575758
2,AmaNEk,32,26,2.0,6,14,5.0,1.230769,0.653061,0.437500
3,nexa,29,21,6.0,8,18,4.0,1.380952,0.591837,0.620690
4,broky,20,15,1.0,5,6,4.0,1.333333,0.408163,0.300000
5,olofmeister,20,16,0.0,4,14,1.0,1.250000,0.408163,0.700000
6,device,16,16,0.0,0,5,2.0,1.000000,0.326531,0.312500
7,huNter,16,13,3.0,3,3,4.0,1.230769,0.326531,0.187500
8,huNter-,16,12,3.0,4,4,2.0,1.333333,0.326531,0.250000
9,hampus,15,17,5.0,-2,7,0.0,0.882353,0.306122,0.466667


In [47]:
g_kill_stats['de_vertigo']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,huNter-,121,116,17.0,5,53,10.0,1.043103,0.703488,0.438017
1,AmaNEk,101,112,16.0,-11,40,10.0,0.901786,0.587209,0.396040
2,JACKZ,99,137,18.0,-38,68,20.0,0.722628,0.575581,0.686869
3,NiKo,82,80,20.0,2,36,17.0,1.025000,0.476744,0.439024
4,nexa,81,115,23.0,-34,38,15.0,0.704348,0.470930,0.469136
5,blameF,56,42,12.0,14,22,12.0,1.333333,0.325581,0.392857
6,k0nfig,53,44,7.0,9,30,10.0,1.204545,0.308140,0.566038
7,NiKo,47,42,4.0,5,24,5.0,1.119048,0.273256,0.510638
8,poizon,39,49,11.0,-10,18,5.0,0.795918,0.226744,0.461538
9,NEOFRAG,32,17,2.0,15,10,4.0,1.882353,0.186047,0.312500


In [48]:
g_kill_stats['de_nuke']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,AmaNEk,144,104,12.0,40,58,23.0,1.384615,0.808989,0.402778
1,nexa,118,104,17.0,14,50,12.0,1.134615,0.662921,0.423729
2,huNter-,115,123,20.0,-8,56,10.0,0.934959,0.646067,0.486957
3,JACKZ,88,125,16.0,-37,62,12.0,0.704000,0.494382,0.704545
4,NiKo,76,70,19.0,6,39,12.0,1.085714,0.426966,0.513158
5,NiKo,47,53,4.0,-6,15,12.0,0.886792,0.264045,0.319149
6,s1mple,42,32,5.0,10,16,8.0,1.312500,0.235955,0.380952
7,electronic,41,41,11.0,0,17,11.0,1.000000,0.230337,0.414634
8,b1t,39,28,4.0,11,26,6.0,1.392857,0.219101,0.666667
9,Perfecto,36,35,8.0,1,18,6.0,1.028571,0.202247,0.500000


In [49]:
g_kill_stats['de_dust2']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,huNter-,153,152,21,1,72,20.0,1.006579,0.634855,0.470588
1,nexa,147,163,23,-16,68,17.0,0.901840,0.609959,0.462585
2,JACKZ,134,167,21,-33,93,29.0,0.802395,0.556017,0.694030
3,NiKo,120,103,21,17,64,20.0,1.165049,0.497925,0.533333
4,AmaNEk,113,148,29,-35,35,13.0,0.763514,0.468880,0.309735
5,NiKo,62,62,15,0,27,10.0,1.000000,0.257261,0.435484
6,syrsoN,59,44,6,15,18,20.0,1.340909,0.244813,0.305085
7,s1mple,57,32,8,25,25,11.0,1.781250,0.236515,0.438596
8,tabseN,56,51,19,5,29,9.0,1.098039,0.232365,0.517857
9,tiziaN,56,40,13,16,38,3.0,1.400000,0.232365,0.678571


In [50]:
hKillData, hRoundData, hDamageData, hFrameData, hPlayerFrames, hBombEvents, hGrenadeData, hFlashData = generateData(generateDemoParsers(getDemoPaths(heroic_path)))

13:16:47 [INFO] Go version>=1.14.0
13:16:47 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Heroic/astralis-vs-heroic-m1-inferno.dem
13:16:47 [INFO] Setting demo id to astralis-vs-heroic-m1-inferno
13:16:47 [INFO] Setting parse rate to 128
13:16:47 [INFO] Go version>=1.14.0
13:16:47 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Heroic/astralis-vs-heroic-m2-overpass.dem
13:16:47 [INFO] Setting demo id to astralis-vs-heroic-m2-overpass
13:16:47 [INFO] Setting parse rate to 128
13:16:48 [INFO] Go version>=1.14.0
13:16:48 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Heroic/faze-vs-heroic-m1-mirage.dem
13:16:48 [INFO] Setting demo id to faze-vs-heroic-m1-mirage
13:16:48 [INFO] Setting parse rate to 128
13:16:48 [INFO] Go version>=1.14.0
13:16:48 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Heroic/faze-vs-heroic-m2-nuke.dem
13:16:48 [INFO] Setting demo id to faze-vs-heroic-m2-nuke
13:16:48 [INFO] Setting parse rate to 128
13:16:

13:16:59 [INFO] Setting parse rate to 128
13:16:59 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:16:59 [INFO] Looking for file at /mnt/d/TOP_10/Heroic/astralis-vs-heroic-m1-inferno.dem


# Demo Parsers: 33


13:17:16 [INFO] Wrote demo parse output to astralis-vs-heroic-m1-inferno.json
13:17:16 [INFO] Reading in JSON from astralis-vs-heroic-m1-inferno.json
13:17:17 [INFO] JSON data loaded, available in the `json` attribute to parser
13:17:17 [INFO] Successfully parsed JSON output
13:17:17 [INFO] Successfully returned JSON output
13:17:17 [INFO] Parsed kills to Pandas DataFrame
13:17:17 [INFO] Parsed rounds to Pandas DataFrame
13:17:17 [INFO] Parsed damages to Pandas DataFrame
13:17:17 [INFO] Parsed frames to Pandas DataFrame
13:17:17 [INFO] Parsed player frames to Pandas DataFrame
13:17:17 [INFO] Parsed bomb_events to Pandas DataFrame
13:17:17 [INFO] Parsed grenades to Pandas DataFrame
13:17:17 [INFO] Parsed flashes to Pandas DataFrame
13:17:17 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:17:17 [INFO] Looking for file at /mnt/d/TOP_10/Heroic/astralis-vs-heroic-m2-overpass.dem
13:17:48 [INFO] Wrote demo parse output to astralis-vs-heroic-m2-overpass.json
13:17

13:20:08 [INFO] Reading in JSON from heroic-vs-bad-news-bears-m1-mirage.json
13:20:08 [INFO] JSON data loaded, available in the `json` attribute to parser
13:20:08 [INFO] Successfully parsed JSON output
13:20:08 [INFO] Successfully returned JSON output
13:20:08 [INFO] Parsed kills to Pandas DataFrame
13:20:08 [INFO] Parsed rounds to Pandas DataFrame
13:20:08 [INFO] Parsed damages to Pandas DataFrame
13:20:08 [INFO] Parsed frames to Pandas DataFrame
13:20:08 [INFO] Parsed player frames to Pandas DataFrame
13:20:08 [INFO] Parsed bomb_events to Pandas DataFrame
13:20:08 [INFO] Parsed grenades to Pandas DataFrame
13:20:08 [INFO] Parsed flashes to Pandas DataFrame
13:20:08 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:20:08 [INFO] Looking for file at /mnt/d/TOP_10/Heroic/heroic-vs-bad-news-bears-m2-nuke.dem
13:20:27 [INFO] Wrote demo parse output to heroic-vs-bad-news-bears-m2-nuke.json
13:20:27 [INFO] Reading in JSON from heroic-vs-bad-news-bears-m2-nuke.json

13:23:23 [INFO] Reading in JSON from heroic-vs-liquid-m1-overpass.json
13:23:24 [INFO] JSON data loaded, available in the `json` attribute to parser
13:23:24 [INFO] Successfully parsed JSON output
13:23:24 [INFO] Successfully returned JSON output
13:23:24 [INFO] Parsed kills to Pandas DataFrame
13:23:24 [INFO] Parsed rounds to Pandas DataFrame
13:23:24 [INFO] Parsed damages to Pandas DataFrame
13:23:24 [INFO] Parsed frames to Pandas DataFrame
13:23:24 [INFO] Parsed player frames to Pandas DataFrame
13:23:24 [INFO] Parsed bomb_events to Pandas DataFrame
13:23:24 [INFO] Parsed grenades to Pandas DataFrame
13:23:24 [INFO] Parsed flashes to Pandas DataFrame
13:23:24 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:23:24 [INFO] Looking for file at /mnt/d/TOP_10/Heroic/heroic-vs-liquid-m2-nuke.dem
13:23:44 [INFO] Wrote demo parse output to heroic-vs-liquid-m2-nuke.json
13:23:44 [INFO] Reading in JSON from heroic-vs-liquid-m2-nuke.json
13:23:44 [INFO] JSON data loa

13:26:31 [INFO] JSON data loaded, available in the `json` attribute to parser
13:26:31 [INFO] Successfully parsed JSON output
13:26:31 [INFO] Successfully returned JSON output
13:26:31 [INFO] Parsed kills to Pandas DataFrame
13:26:31 [INFO] Parsed rounds to Pandas DataFrame
13:26:31 [INFO] Parsed damages to Pandas DataFrame
13:26:31 [INFO] Parsed frames to Pandas DataFrame
13:26:31 [INFO] Parsed player frames to Pandas DataFrame
13:26:31 [INFO] Parsed bomb_events to Pandas DataFrame
13:26:31 [INFO] Parsed grenades to Pandas DataFrame
13:26:31 [INFO] Parsed flashes to Pandas DataFrame
13:26:31 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:26:31 [INFO] Looking for file at /mnt/d/TOP_10/Heroic/natus-vincere-vs-heroic-m1-mirage.dem
13:26:50 [INFO] Wrote demo parse output to natus-vincere-vs-heroic-m1-mirage.json
13:26:50 [INFO] Reading in JSON from natus-vincere-vs-heroic-m1-mirage.json
13:26:50 [INFO] JSON data loaded, available in the `json` attribute to pa

In [51]:
finalhKillData = concat_data(hKillData)
finalhRoundData = concat_data(hRoundData)
finalhDamageData = concat_data(hDamageData)
finalhGrenadeData = concat_data(hGrenadeData)
finalhFlashData = concat_data(hFlashData)
finalhBombData = concat_data(hBombEvents)

In [52]:
hkillMapData = divideDataByMaps(finalhKillData)
hroundMapData = divideDataByMaps(finalhRoundData)
hdamageMapData = divideDataByMaps(finalhDamageData)
hgrenadeMapData = divideDataByMaps(finalhGrenadeData)
hflashMapData = divideDataByMaps(finalhFlashData)
hvBombMapData = divideDataByMaps(finalhBombData)

In [56]:
h_kill_stats = {}
for x in hkillMapData.keys():
    h_kill_stats[x] = stats.kill_stats(hkillMapData[x], hroundMapData[x])

In [57]:
h_kill_stats['de_mirage']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,stavn,108,109,19.0,-1,50,23.0,0.990826,0.679245,0.462963
1,TeSeS,106,100,15.0,6,66,11.0,1.060000,0.666667,0.622642
2,sjuush,99,113,15.0,-14,66,21.0,0.876106,0.622642,0.666667
3,refrezh,96,107,16.0,-11,54,13.0,0.897196,0.603774,0.562500
4,cadiaN,90,104,15.0,-14,22,11.0,0.865385,0.566038,0.244444
5,s1mple,30,17,4.0,13,10,6.0,1.764706,0.188679,0.333333
6,Plopski,29,20,4.0,9,16,5.0,1.450000,0.182390,0.551724
7,Snappi,29,9,0.0,20,20,3.0,3.222222,0.182390,0.689655
8,oBo,28,20,9.0,8,11,4.0,1.400000,0.176101,0.392857
9,LNZ,27,19,4.0,8,16,3.0,1.421053,0.169811,0.592593


In [58]:
h_kill_stats['de_overpass']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,refrezh,105,96,22.0,9,57,12.0,1.093750,0.729167,0.542857
1,TeSeS,97,105,15.0,-8,61,17.0,0.923810,0.673611,0.628866
2,sjuush,96,97,20.0,-1,46,15.0,0.989691,0.666667,0.479167
3,stavn,93,95,18.0,-2,39,17.0,0.978947,0.645833,0.419355
4,cadiaN,83,94,11.0,-11,19,13.0,0.882979,0.576389,0.228916
5,s1mple,32,18,2.0,14,11,5.0,1.777778,0.222222,0.343750
6,dupreeh,32,16,3.0,16,18,3.0,2.000000,0.222222,0.562500
7,Stewie2K,27,19,4.0,8,13,6.0,1.421053,0.187500,0.481481
8,hades,27,20,3.0,7,7,2.0,1.350000,0.187500,0.259259
9,NAF,24,18,8.0,6,15,2.0,1.333333,0.166667,0.625000


In [59]:
h_kill_stats['de_inferno']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,refrezh,131,102,18.0,29,74,13.0,1.284314,0.744318,0.564885
1,stavn,129,119,31.0,10,47,28.0,1.084034,0.732955,0.364341
2,TeSeS,123,115,29.0,8,60,19.0,1.069565,0.698864,0.487805
3,cadiaN,113,104,13.0,9,24,21.0,1.086538,0.642045,0.212389
4,sjuush,86,99,29.0,-13,50,15.0,0.868687,0.488636,0.581395
5,dupreeh,35,33,4.0,2,18,6.0,1.060606,0.198864,0.514286
6,Bubzkji,34,35,3.0,-1,12,4.0,0.971429,0.193182,0.352941
7,MICHU,31,24,4.0,7,13,4.0,1.291667,0.176136,0.419355
8,NAF,31,19,3.0,12,13,4.0,1.631579,0.176136,0.419355
9,stanislaw,27,23,1.0,4,13,3.0,1.173913,0.153409,0.481481


In [60]:
h_kill_stats['de_ancient']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,cadiaN,104,70,9.0,34,23,15,1.485714,0.818898,0.221154
1,TeSeS,95,87,19.0,8,48,11,1.091954,0.748031,0.505263
2,sjuush,90,84,19.0,6,47,11,1.071429,0.708661,0.522222
3,stavn,77,96,15.0,-19,33,8,0.802083,0.606299,0.428571
4,refrezh,70,86,13.0,-16,33,9,0.813953,0.551181,0.471429
5,mir,39,28,6.0,11,16,5,1.392857,0.307087,0.410256
6,degster,38,23,4.0,15,9,5,1.652174,0.299213,0.236842
7,KSCERATO,29,21,7.0,8,14,6,1.380952,0.228346,0.482759
8,SHOOWTiME,26,22,5.0,4,14,1,1.181818,0.204724,0.538462
9,chopper,24,29,4.0,-5,10,4,0.827586,0.188976,0.416667


In [61]:
h_kill_stats['de_vertigo']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,refrezh,44,26,7,18,18,2.0,1.692308,0.936170,0.409091
1,cadiaN,41,25,4,16,12,12.0,1.640000,0.872340,0.292683
2,TeSeS,37,27,9,10,26,8.0,1.370370,0.787234,0.702703
3,sjuush,33,21,3,12,15,4.0,1.571429,0.702128,0.454545
4,stavn,30,28,10,2,12,5.0,1.071429,0.638298,0.400000
5,Magisk,18,20,5,-2,7,1.0,0.900000,0.382979,0.388889
6,apEX,16,20,2,-4,10,2.0,0.800000,0.340426,0.625000
7,dupreeh,15,16,2,-1,9,1.0,0.937500,0.319149,0.600000
8,Kyojin,14,19,4,-5,8,3.0,0.736842,0.297872,0.571429
9,ZywOo,13,19,3,-6,5,2.0,0.684211,0.276596,0.384615


In [62]:
h_kill_stats['de_nuke']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,stavn,189,186,50.0,3,98.0,35.0,1.016129,0.702602,0.518519
1,TeSeS,186,171,20.0,15,107.0,28.0,1.087719,0.691450,0.575269
2,sjuush,186,164,27.0,22,106.0,19.0,1.134146,0.691450,0.569892
3,cadiaN,175,153,30.0,22,38.0,35.0,1.143791,0.650558,0.217143
4,refrezh,170,175,27.0,-5,71.0,27.0,0.971429,0.631970,0.417647
5,s1mple,31,19,3.0,12,20.0,6.0,1.631579,0.115242,0.645161
6,device,27,10,3.0,17,4.0,3.0,2.700000,0.100372,0.148148
7,degster,27,13,0.0,14,6.0,3.0,2.076923,0.100372,0.222222
8,ZywOo,26,14,2.0,12,9.0,7.0,1.857143,0.096654,0.346154
9,oBo,26,21,4.0,5,8.0,5.0,1.238095,0.096654,0.307692


In [63]:
h_kill_stats['de_dust2']

KeyError: 'de_dust2'

In [64]:
nKillData, nRoundData, nDamageData, nFrameData, nPlayerFrames, nBombEvents, nGrenadeData, nFlashData = generateData(generateDemoParsers(getDemoPaths(nip_path)))

13:29:43 [INFO] Go version>=1.14.0
13:29:43 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/NiP/g2-vs-nip-m1-ancient.dem
13:29:43 [INFO] Setting demo id to g2-vs-nip-m1-ancient
13:29:43 [INFO] Setting parse rate to 128
13:29:43 [INFO] Go version>=1.14.0
13:29:43 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/NiP/g2-vs-nip-m2-mirage.dem
13:29:43 [INFO] Setting demo id to g2-vs-nip-m2-mirage
13:29:43 [INFO] Setting parse rate to 128
13:29:43 [INFO] Go version>=1.14.0
13:29:43 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/NiP/g2-vs-nip-m3-inferno.dem
13:29:43 [INFO] Setting demo id to g2-vs-nip-m3-inferno
13:29:43 [INFO] Setting parse rate to 128
13:29:44 [INFO] Go version>=1.14.0
13:29:44 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/NiP/gambit-vs-nip-m1-ancient.dem
13:29:44 [INFO] Setting demo id to gambit-vs-nip-m1-ancient
13:29:44 [INFO] Setting parse rate to 128
13:29:44 [INFO] Go version>=1.14.0
13:29:44 [INFO] Initialized

# Demo Parsers: 32


13:30:08 [INFO] Wrote demo parse output to g2-vs-nip-m1-ancient.json
13:30:08 [INFO] Reading in JSON from g2-vs-nip-m1-ancient.json
13:30:08 [INFO] JSON data loaded, available in the `json` attribute to parser
13:30:08 [INFO] Successfully parsed JSON output
13:30:08 [INFO] Successfully returned JSON output
13:30:08 [INFO] Parsed kills to Pandas DataFrame
13:30:08 [INFO] Parsed rounds to Pandas DataFrame
13:30:08 [INFO] Parsed damages to Pandas DataFrame
13:30:08 [INFO] Parsed frames to Pandas DataFrame
13:30:08 [INFO] Parsed player frames to Pandas DataFrame
13:30:08 [INFO] Parsed bomb_events to Pandas DataFrame
13:30:08 [INFO] Parsed grenades to Pandas DataFrame
13:30:08 [INFO] Parsed flashes to Pandas DataFrame
13:30:08 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:30:08 [INFO] Looking for file at /mnt/d/TOP_10/NiP/g2-vs-nip-m2-mirage.dem
13:30:27 [INFO] Wrote demo parse output to g2-vs-nip-m2-mirage.json
13:30:27 [INFO] Reading in JSON from g2-vs-nip-m

13:32:57 [INFO] JSON data loaded, available in the `json` attribute to parser
13:32:57 [INFO] Successfully parsed JSON output
13:32:57 [INFO] Successfully returned JSON output
13:32:57 [INFO] Parsed kills to Pandas DataFrame
13:32:57 [INFO] Parsed rounds to Pandas DataFrame
13:32:58 [INFO] Parsed damages to Pandas DataFrame
13:32:58 [INFO] Parsed frames to Pandas DataFrame
13:32:58 [INFO] Parsed player frames to Pandas DataFrame
13:32:58 [INFO] Parsed bomb_events to Pandas DataFrame
13:32:58 [INFO] Parsed grenades to Pandas DataFrame
13:32:58 [INFO] Parsed flashes to Pandas DataFrame
13:32:58 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:32:58 [INFO] Looking for file at /mnt/d/TOP_10/NiP/mousesports-vs-nip-m3-inferno.dem
13:33:14 [INFO] Wrote demo parse output to mousesports-vs-nip-m3-inferno.json
13:33:14 [INFO] Reading in JSON from mousesports-vs-nip-m3-inferno.json
13:33:14 [INFO] JSON data loaded, available in the `json` attribute to parser
13:33:14 [

13:35:21 [INFO] Parsed kills to Pandas DataFrame
13:35:21 [INFO] Parsed rounds to Pandas DataFrame
13:35:21 [INFO] Parsed damages to Pandas DataFrame
13:35:21 [INFO] Parsed frames to Pandas DataFrame
13:35:22 [INFO] Parsed player frames to Pandas DataFrame
13:35:22 [INFO] Parsed bomb_events to Pandas DataFrame
13:35:22 [INFO] Parsed grenades to Pandas DataFrame
13:35:22 [INFO] Parsed flashes to Pandas DataFrame
13:35:22 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:35:22 [INFO] Looking for file at /mnt/d/TOP_10/NiP/nip-vs-furia-m3-mirage.dem
13:35:34 [INFO] Wrote demo parse output to nip-vs-furia-m3-mirage.json
13:35:34 [INFO] Reading in JSON from nip-vs-furia-m3-mirage.json
13:35:35 [INFO] JSON data loaded, available in the `json` attribute to parser
13:35:35 [INFO] Successfully parsed JSON output
13:35:35 [INFO] Successfully returned JSON output
13:35:35 [INFO] Parsed kills to Pandas DataFrame
13:35:35 [INFO] Parsed rounds to Pandas DataFrame
13:35:35 [

13:37:54 [INFO] Parsed frames to Pandas DataFrame
13:37:54 [INFO] Parsed player frames to Pandas DataFrame
13:37:54 [INFO] Parsed bomb_events to Pandas DataFrame
13:37:54 [INFO] Parsed grenades to Pandas DataFrame
13:37:54 [INFO] Parsed flashes to Pandas DataFrame
13:37:54 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:37:54 [INFO] Looking for file at /mnt/d/TOP_10/NiP/nip-vs-virtus-pro-m2-inferno.dem
13:38:11 [INFO] Wrote demo parse output to nip-vs-virtus-pro-m2-inferno.json
13:38:11 [INFO] Reading in JSON from nip-vs-virtus-pro-m2-inferno.json
13:38:12 [INFO] JSON data loaded, available in the `json` attribute to parser
13:38:12 [INFO] Successfully parsed JSON output
13:38:12 [INFO] Successfully returned JSON output
13:38:12 [INFO] Parsed kills to Pandas DataFrame
13:38:12 [INFO] Parsed rounds to Pandas DataFrame
13:38:12 [INFO] Parsed damages to Pandas DataFrame
13:38:12 [INFO] Parsed frames to Pandas DataFrame
13:38:13 [INFO] Parsed player frames to P

In [65]:
finalnKillData = concat_data(nKillData)
finalnRoundData = concat_data(nRoundData)
finalnDamageData = concat_data(nDamageData)
finalnGrenadeData = concat_data(nGrenadeData)
finalnFlashData = concat_data(nFlashData)
finalnBombData = concat_data(nBombEvents)

In [66]:
nkillMapData = divideDataByMaps(finalnKillData)
nroundMapData = divideDataByMaps(finalnRoundData)
ndamageMapData = divideDataByMaps(finalnDamageData)
ngrenadeMapData = divideDataByMaps(finalnGrenadeData)
nflashMapData = divideDataByMaps(finalnFlashData)
nBombMapData = divideDataByMaps(finalnBombData)

In [67]:
n_kill_stats = {}
for x in nkillMapData.keys():
    n_kill_stats[x] = stats.kill_stats(nkillMapData[x], nroundMapData[x])

In [68]:
n_kill_stats['de_mirage']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,device,108,79,15.0,29,46,15.0,1.367089,0.830769,0.425926
1,REZ,98,94,22.0,4,59,14.0,1.042553,0.753846,0.602041
2,hampus,97,92,18.0,5,42,21.0,1.054348,0.746154,0.432990
3,Plopski,80,77,9.0,3,26,8.0,1.038961,0.615385,0.325000
4,LNZ,71,85,19.0,-14,47,10.0,0.835294,0.546154,0.661972
5,NiKo,33,19,4.0,14,18,4.0,1.736842,0.253846,0.545455
6,syrsoN,26,22,2.0,4,6,7.0,1.181818,0.200000,0.230769
7,flameZ,26,15,1.0,11,13,4.0,1.733333,0.200000,0.500000
8,valde,23,15,4.0,8,14,4.0,1.533333,0.176923,0.608696
9,VINI,23,11,3.0,12,6,4.0,2.090909,0.176923,0.260870


In [69]:
n_kill_stats['de_overpass']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,REZ,88,65,6,23,51,10,1.353846,0.846154,0.579545
1,device,80,64,13,16,24,13,1.250000,0.769231,0.300000
2,Plopski,76,67,15,9,33,8,1.134328,0.730769,0.434211
3,hampus,62,68,13,-6,21,11,0.911765,0.596154,0.338710
4,LNZ,50,66,17,-16,26,4,0.757576,0.480769,0.520000
5,Grim,31,23,7,8,14,7,1.347826,0.298077,0.451613
6,malbsMd,30,28,4,2,18,5,1.071429,0.288462,0.600000
7,xns,30,22,5,8,19,5,1.363636,0.288462,0.633333
8,NAF,25,25,9,0,13,2,1.000000,0.240385,0.520000
9,flameZ,25,24,5,1,10,6,1.041667,0.240385,0.400000


In [70]:
n_kill_stats['de_inferno']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,REZ,139,125,33.0,14,69,16.0,1.112000,0.743316,0.496403
1,hampus,136,128,28.0,8,62,27.0,1.062500,0.727273,0.455882
2,device,121,108,22.0,13,45,25.0,1.120370,0.647059,0.371901
3,Plopski,115,107,28.0,8,57,15.0,1.074766,0.614973,0.495652
4,LNZ,105,105,25.0,0,56,8.0,1.000000,0.561497,0.533333
5,Ax1Le,35,22,3.0,13,21,3.0,1.590909,0.187166,0.600000
6,Hobbit,33,24,4.0,9,20,4.0,1.375000,0.176471,0.606061
7,nafany,32,31,4.0,1,15,9.0,1.032258,0.171123,0.468750
8,SH1R0,29,25,7.0,4,6,7.0,1.160000,0.155080,0.206897
9,syrsoN,28,33,1.0,-5,13,8.0,0.848485,0.149733,0.464286


In [71]:
n_kill_stats['de_ancient']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,REZ,134,117,15.0,17,57,19,1.145299,0.724324,0.425373
1,hampus,131,138,40.0,-7,69,28,0.949275,0.708108,0.526718
2,device,123,120,24.0,3,40,17,1.025000,0.664865,0.325203
3,Plopski,107,116,28.0,-9,45,15,0.922414,0.578378,0.420561
4,LNZ,94,117,23.0,-23,55,7,0.803419,0.508108,0.585106
5,Hobbit,37,34,6.0,3,25,6,1.088235,0.200000,0.675676
6,SH1R0,36,24,4.0,12,9,9,1.500000,0.194595,0.250000
7,Ax1Le,34,35,9.0,-1,15,3,0.971429,0.183784,0.441176
8,KSCERATO,32,18,4.0,14,10,6,1.777778,0.172973,0.312500
9,nafany,27,41,10.0,-14,12,6,0.658537,0.145946,0.444444


In [72]:
n_kill_stats['de_vertigo']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,ropz,20,9,1.0,11,12,2.0,2.222222,0.909091,0.600000
1,acoR,19,6,4.0,13,3,4.0,3.166667,0.863636,0.157895
2,dexter,19,17,5.0,2,9,5.0,1.117647,0.863636,0.473684
3,frozen,17,8,0.0,9,10,4.0,2.125000,0.772727,0.588235
4,Bymas,16,11,2.0,5,6,2.0,1.454545,0.727273,0.375000
5,REZ,14,18,3.0,-4,7,0.0,0.777778,0.636364,0.500000
6,LNZ,10,17,2.0,-7,5,2.0,0.588235,0.454545,0.500000
7,hampus,10,21,5.0,-11,5,1.0,0.476190,0.454545,0.500000
8,Plopski,8,18,1.0,-10,3,1.0,0.444444,0.363636,0.375000
9,device,8,17,3.0,-9,3,1.0,0.470588,0.363636,0.375000


In [73]:
n_kill_stats['de_nuke']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,device,43,37,4.0,6,16,5.0,1.162162,0.767857,0.372093
1,REZ,41,37,6.0,4,18,3.0,1.108108,0.732143,0.439024
2,LNZ,37,40,4.0,-3,18,2.0,0.925000,0.660714,0.486486
3,Plopski,34,33,6.0,1,16,9.0,1.030303,0.607143,0.470588
4,hampus,34,42,10.0,-8,18,4.0,0.809524,0.607143,0.529412
5,malbsMd,26,21,4.0,5,15,3.0,1.238095,0.464286,0.576923
6,yuurih,25,17,3.0,8,9,6.0,1.470588,0.446429,0.360000
7,arT,24,21,2.0,3,9,6.0,1.142857,0.428571,0.375000
8,KSCERATO,21,12,3.0,9,6,0.0,1.750000,0.375000,0.285714
9,Maluk3,18,21,4.0,-3,2,5.0,0.857143,0.321429,0.111111


In [74]:
n_kill_stats['de_dust2']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,hampus,110,106,21.0,4,48,22.0,1.037736,0.748299,0.436364
1,REZ,91,103,14.0,-12,46,17.0,0.883495,0.619048,0.505495
2,device,91,88,12.0,3,20,16.0,1.034091,0.619048,0.219780
3,LNZ,78,90,10.0,-12,41,3.0,0.866667,0.530612,0.525641
4,Plopski,74,101,15.0,-27,36,8.0,0.732673,0.503401,0.486486
5,EliGE,40,30,6.0,10,23,4.0,1.333333,0.272109,0.575000
6,Krad,30,17,1.0,13,11,4.0,1.764706,0.204082,0.366667
7,FalleN,30,29,6.0,1,7,12.0,1.034483,0.204082,0.233333
8,NAF,26,32,5.0,-6,11,4.0,0.812500,0.176871,0.423077
9,Grim,24,32,4.0,-8,11,5.0,0.750000,0.163265,0.458333


In [11]:
vKillData, vRoundData, vDamageData, vFrameData, vPlayerFrames, vBombEvents, vGrenadeData, vFlashData = generateData(generateDemoParsers(getDemoPaths(vp_path)))

12:35:40 [INFO] Go version>=1.14.0
12:35:40 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/VP/100pg-vs-virtus-pro-m1-dust2.dem
12:35:40 [INFO] Setting demo id to 100pg-vs-virtus-pro-m1-dust2
12:35:40 [INFO] Setting parse rate to 128
12:35:40 [INFO] Go version>=1.14.0
12:35:40 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/VP/100pg-vs-virtus-pro-m2-inferno.dem
12:35:40 [INFO] Setting demo id to 100pg-vs-virtus-pro-m2-inferno
12:35:40 [INFO] Setting parse rate to 128
12:35:40 [INFO] Go version>=1.14.0
12:35:40 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/VP/100pg-vs-virtus-pro-m3-overpass.dem
12:35:40 [INFO] Setting demo id to 100pg-vs-virtus-pro-m3-overpass
12:35:40 [INFO] Setting parse rate to 128
12:35:40 [INFO] Go version>=1.14.0
12:35:40 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/VP/big-vs-virtus-pro-m1-dust2.dem
12:35:40 [INFO] Setting demo id to big-vs-virtus-pro-m1-dust2
12:35:40 [INFO] Setting parse rate to 128
12

12:35:45 [INFO] Setting parse rate to 128
12:35:45 [INFO] Go version>=1.14.0
12:35:45 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/VP/virtus-pro-vs-sinners-m1-vertigo.dem
12:35:45 [INFO] Setting demo id to virtus-pro-vs-sinners-m1-vertigo
12:35:45 [INFO] Setting parse rate to 128
12:35:45 [INFO] Go version>=1.14.0
12:35:45 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/VP/virtus-pro-vs-sinners-m2-inferno.dem
12:35:45 [INFO] Setting demo id to virtus-pro-vs-sinners-m2-inferno
12:35:45 [INFO] Setting parse rate to 128
12:35:45 [INFO] Go version>=1.14.0
12:35:45 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/VP/virtus-pro-vs-spirit-m1-inferno.dem
12:35:45 [INFO] Setting demo id to virtus-pro-vs-spirit-m1-inferno
12:35:45 [INFO] Setting parse rate to 128
12:35:45 [INFO] Go version>=1.14.0
12:35:45 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/VP/virtus-pro-vs-spirit-m2-dust2.dem
12:35:45 [INFO] Setting demo id to virtus-pro-vs-

# Demo Parsers: 38


12:36:07 [INFO] Wrote demo parse output to 100pg-vs-virtus-pro-m1-dust2.json
12:36:07 [INFO] Reading in JSON from 100pg-vs-virtus-pro-m1-dust2.json
12:36:08 [INFO] JSON data loaded, available in the `json` attribute to parser
12:36:08 [INFO] Successfully parsed JSON output
12:36:08 [INFO] Successfully returned JSON output
12:36:08 [INFO] Parsed kills to Pandas DataFrame
12:36:08 [INFO] Parsed rounds to Pandas DataFrame
12:36:08 [INFO] Parsed damages to Pandas DataFrame
12:36:08 [INFO] Parsed frames to Pandas DataFrame
12:36:08 [INFO] Parsed player frames to Pandas DataFrame
12:36:08 [INFO] Parsed bomb_events to Pandas DataFrame
12:36:08 [INFO] Parsed grenades to Pandas DataFrame
12:36:08 [INFO] Parsed flashes to Pandas DataFrame
12:36:08 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
12:36:08 [INFO] Looking for file at /mnt/d/TOP_10/VP/100pg-vs-virtus-pro-m2-inferno.dem
12:36:27 [INFO] Wrote demo parse output to 100pg-vs-virtus-pro-m2-inferno.json
12:36:27 [I

12:40:00 [INFO] Wrote demo parse output to entropiq-vs-virtus-pro-m2-dust2.json
12:40:00 [INFO] Reading in JSON from entropiq-vs-virtus-pro-m2-dust2.json
12:40:01 [INFO] JSON data loaded, available in the `json` attribute to parser
12:40:01 [INFO] Successfully parsed JSON output
12:40:01 [INFO] Successfully returned JSON output
12:40:01 [INFO] Parsed kills to Pandas DataFrame
12:40:01 [INFO] Parsed rounds to Pandas DataFrame
12:40:01 [INFO] Parsed damages to Pandas DataFrame
12:40:01 [INFO] Parsed frames to Pandas DataFrame
12:40:01 [INFO] Parsed player frames to Pandas DataFrame
12:40:01 [INFO] Parsed bomb_events to Pandas DataFrame
12:40:01 [INFO] Parsed grenades to Pandas DataFrame
12:40:01 [INFO] Parsed flashes to Pandas DataFrame
12:40:01 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
12:40:01 [INFO] Looking for file at /mnt/d/TOP_10/VP/g2-vs-virtus-pro-m1-dust2.dem
12:40:19 [INFO] Wrote demo parse output to g2-vs-virtus-pro-m1-dust2.json
12:40:19 [INFO]

12:43:01 [INFO] Reading in JSON from skade-vs-virtus-pro-m1-overpass.json
12:43:02 [INFO] JSON data loaded, available in the `json` attribute to parser
12:43:02 [INFO] Successfully parsed JSON output
12:43:02 [INFO] Successfully returned JSON output
12:43:02 [INFO] Parsed kills to Pandas DataFrame
12:43:02 [INFO] Parsed rounds to Pandas DataFrame
12:43:02 [INFO] Parsed damages to Pandas DataFrame
12:43:02 [INFO] Parsed frames to Pandas DataFrame
12:43:02 [INFO] Parsed player frames to Pandas DataFrame
12:43:02 [INFO] Parsed bomb_events to Pandas DataFrame
12:43:02 [INFO] Parsed grenades to Pandas DataFrame
12:43:02 [INFO] Parsed flashes to Pandas DataFrame
12:43:02 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
12:43:02 [INFO] Looking for file at /mnt/d/TOP_10/VP/skade-vs-virtus-pro-m2-dust2.dem
12:43:15 [INFO] Wrote demo parse output to skade-vs-virtus-pro-m2-dust2.json
12:43:15 [INFO] Reading in JSON from skade-vs-virtus-pro-m2-dust2.json
12:43:15 [INFO] JS

12:45:59 [INFO] Reading in JSON from virtus-pro-vs-faze-m1-mirage.json
12:45:59 [INFO] JSON data loaded, available in the `json` attribute to parser
12:45:59 [INFO] Successfully parsed JSON output
12:45:59 [INFO] Successfully returned JSON output
12:45:59 [INFO] Parsed kills to Pandas DataFrame
12:45:59 [INFO] Parsed rounds to Pandas DataFrame
12:45:59 [INFO] Parsed damages to Pandas DataFrame
12:45:59 [INFO] Parsed frames to Pandas DataFrame
12:45:59 [INFO] Parsed player frames to Pandas DataFrame
12:45:59 [INFO] Parsed bomb_events to Pandas DataFrame
12:45:59 [INFO] Parsed grenades to Pandas DataFrame
12:45:59 [INFO] Parsed flashes to Pandas DataFrame
12:45:59 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
12:45:59 [INFO] Looking for file at /mnt/d/TOP_10/VP/virtus-pro-vs-faze-m2-inferno.dem
12:46:13 [INFO] Wrote demo parse output to virtus-pro-vs-faze-m2-inferno.json
12:46:13 [INFO] Reading in JSON from virtus-pro-vs-faze-m2-inferno.json
12:46:13 [INFO] JS

12:48:11 [INFO] Reading in JSON from virtus-pro-vs-spirit-m2-dust2.json
12:48:12 [INFO] JSON data loaded, available in the `json` attribute to parser
12:48:12 [INFO] Successfully parsed JSON output
12:48:12 [INFO] Successfully returned JSON output
12:48:12 [INFO] Parsed kills to Pandas DataFrame
12:48:12 [INFO] Parsed rounds to Pandas DataFrame
12:48:12 [INFO] Parsed damages to Pandas DataFrame
12:48:12 [INFO] Parsed frames to Pandas DataFrame
12:48:12 [INFO] Parsed player frames to Pandas DataFrame
12:48:12 [INFO] Parsed bomb_events to Pandas DataFrame
12:48:12 [INFO] Parsed grenades to Pandas DataFrame
12:48:12 [INFO] Parsed flashes to Pandas DataFrame
12:48:12 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
12:48:12 [INFO] Looking for file at /mnt/d/TOP_10/VP/virtus-pro-vs-spirit-m3-overpass.dem
12:48:38 [INFO] Wrote demo parse output to virtus-pro-vs-spirit-m3-overpass.json
12:48:38 [INFO] Reading in JSON from virtus-pro-vs-spirit-m3-overpass.json
12:48:38

In [14]:
finalvKillData = concat_data(vKillData)
finalvRoundData = concat_data(vRoundData)
finalvDamageData = concat_data(vDamageData)
finalvGrenadeData = concat_data(vGrenadeData)
finalvFlashData = concat_data(vFlashData)
finalvBombData = concat_data(vBombEvents)

In [15]:
vkillMapData = divideDataByMaps(finalvKillData)
vroundMapData = divideDataByMaps(finalvRoundData)
vdamageMapData = divideDataByMaps(finalvDamageData)
vgrenadeMapData = divideDataByMaps(finalvGrenadeData)
vflashMapData = divideDataByMaps(finalvFlashData)
vBombMapData = divideDataByMaps(finalvBombData)

In [16]:
v_kill_stats = {}
for x in vkillMapData.keys():
    v_kill_stats[x] = stats.kill_stats(vkillMapData[x], vroundMapData[x])

In [17]:
v_kill_stats['de_mirage']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,YEKINDAR,89,78,19.0,11,58,23.0,1.141026,0.760684,0.651685
1,Jame,80,58,12.0,22,22,15.0,1.379310,0.683761,0.275000
2,buster,76,72,14.0,4,39,4.0,1.055556,0.649573,0.513158
3,SANJI,64,73,9.0,-9,34,7.0,0.876712,0.547009,0.531250
4,qikert,53,71,10.0,-18,28,7.0,0.746479,0.452991,0.528302
5,blameF,43,29,4.0,14,23,9.0,1.482759,0.367521,0.534884
6,jks,34,27,8.0,7,19,4.0,1.259259,0.290598,0.558824
7,poizon,30,31,5.0,-1,9,8.0,0.967742,0.256410,0.300000
8,Jerry,26,14,0.0,12,17,9.0,1.857143,0.222222,0.653846
9,NiKo,25,21,6.0,4,11,9.0,1.190476,0.213675,0.440000


In [18]:
v_kill_stats['de_overpass']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Jame,151,126,15.0,25,32,24.0,1.198413,0.722488,0.211921
1,YEKINDAR,138,156,20.0,-18,86,33.0,0.884615,0.660287,0.623188
2,buster,130,138,28.0,-8,75,22.0,0.942029,0.622010,0.576923
3,SANJI,128,149,28.0,-21,71,11.0,0.859060,0.612440,0.554688
4,Qikert,83,105,23.0,-22,55,7.0,0.790476,0.397129,0.662651
5,qikert,45,42,10.0,3,27,0.0,1.071429,0.215311,0.600000
6,icem4N,32,23,4.0,9,19,8.0,1.391304,0.153110,0.593750
7,w1nt3r,32,25,5.0,7,12,3.0,1.280000,0.153110,0.375000
8,Bubzkji,30,23,4.0,7,16,4.0,1.304348,0.143541,0.533333
9,smiley,27,20,2.0,7,9,3.0,1.350000,0.129187,0.333333


In [19]:
v_kill_stats['de_inferno']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Jame,158,132,23.0,26,32,18.0,1.196970,0.634538,0.202532
1,buster,153,159,24.0,-6,65,25.0,0.962264,0.614458,0.424837
2,YEKINDAR,147,172,37.0,-25,86,39.0,0.854651,0.590361,0.585034
3,SANJI,143,164,24.0,-21,84,13.0,0.871951,0.574297,0.587413
4,qikert,110,94,18.0,16,66,21.0,1.170213,0.441767,0.600000
5,Qikert,73,72,12.0,1,48,10.0,1.013889,0.293173,0.657534
6,tabseN,45,41,8.0,4,28,6.0,1.097561,0.180723,0.622222
7,syrsoN,41,34,2.0,7,7,5.0,1.205882,0.164659,0.170732
8,XANTARES,40,43,12.0,-3,20,7.0,0.930233,0.160643,0.500000
9,tiziaN,40,42,10.0,-2,22,6.0,0.952381,0.160643,0.550000


In [20]:
v_kill_stats['de_ancient']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,REZ,22,17,0.0,5,11,1,1.294118,0.88,0.500000
1,buster,22,17,5.0,5,10,2,1.294118,0.88,0.454545
2,Jame,19,15,2.0,4,4,4,1.266667,0.76,0.210526
3,SANJI,19,12,3.0,7,8,1,1.583333,0.76,0.421053
4,hampus,17,18,4.0,-1,12,4,0.944444,0.68,0.705882
5,YEKINDAR,15,15,5.0,0,9,5,1.000000,0.60,0.600000
6,Plopski,14,18,3.0,-4,5,4,0.777778,0.56,0.357143
7,device,13,17,2.0,-4,4,1,0.764706,0.52,0.307692
8,qikert,13,14,3.0,-1,5,2,0.928571,0.52,0.384615
9,LNZ,7,19,1.0,-12,4,1,0.368421,0.28,0.571429


In [21]:
v_kill_stats['de_vertigo']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,YEKINDAR,47,23,10.0,24,25,11.0,2.043478,1.000000,0.531915
1,Jame,39,21,2.0,18,9,2.0,1.857143,0.829787,0.230769
2,qikert,35,25,6.0,10,16,3.0,1.400000,0.744681,0.457143
3,SANJI,30,29,10.0,1,16,4.0,1.034483,0.638298,0.533333
4,buster,28,32,5.0,-4,14,7.0,0.875000,0.595745,0.500000
5,AmaNEk,21,16,2.0,5,8,2.0,1.312500,0.446809,0.380952
6,oskar,20,17,2.0,3,5,4.0,1.176471,0.425532,0.250000
7,NEOFRAG,15,17,3.0,-2,5,3.0,0.882353,0.319149,0.333333
8,SHOCK,13,19,5.0,-6,6,1.0,0.684211,0.276596,0.461538
9,huNter-,13,18,3.0,-5,5,1.0,0.722222,0.276596,0.384615


In [22]:
v_kill_stats['de_nuke']

KeyError: 'de_nuke'

In [23]:
v_kill_stats['de_dust2']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,YEKINDAR,286,263,55.0,23,181.0,62.0,1.087452,0.756614,0.632867
1,Jame,285,188,35.0,97,74.0,46.0,1.515957,0.753968,0.259649
2,buster,254,224,31.0,30,106.0,34.0,1.133929,0.671958,0.417323
3,SANJI,158,240,49.0,-82,91.0,17.0,0.658333,0.417989,0.575949
4,qikert,126,151,29.0,-25,78.0,20.0,0.834437,0.333333,0.619048
...,...,...,...,...,...,...,...,...,...,...
58,w1nt3r,9,18,3.0,-9,0.0,1.0,0.500000,0.023810,0.000000
59,mantuu,8,8,1.0,0,2.0,2.0,1.000000,0.021164,0.250000
60,Dennyslaw,7,18,1.0,-11,4.0,1.0,0.388889,0.018519,0.571429
61,n0rb3r7_1xBET,6,19,1.0,-13,1.0,1.0,0.315789,0.015873,0.166667


In [75]:
aKillData, aRoundData, aDamageData, aFrameData, aPlayerFrames, aBombEvents, aGrenadeData, aFlashData = generateData(generateDemoParsers(getDemoPaths(astralis_path)))

13:39:27 [INFO] Go version>=1.14.0
13:39:27 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Astralis/astralis-vs-bad-news-bears-m1-inferno.dem
13:39:27 [INFO] Setting demo id to astralis-vs-bad-news-bears-m1-inferno
13:39:27 [INFO] Setting parse rate to 128
13:39:28 [INFO] Go version>=1.14.0
13:39:28 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Astralis/astralis-vs-bad-news-bears-m2-nuke.dem
13:39:28 [INFO] Setting demo id to astralis-vs-bad-news-bears-m2-nuke
13:39:28 [INFO] Setting parse rate to 128
13:39:28 [INFO] Go version>=1.14.0
13:39:28 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Astralis/astralis-vs-ence-m1-ancient.dem
13:39:28 [INFO] Setting demo id to astralis-vs-ence-m1-ancient
13:39:28 [INFO] Setting parse rate to 128
13:39:29 [INFO] Go version>=1.14.0
13:39:29 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Astralis/astralis-vs-ence-m2-nuke.dem
13:39:29 [INFO] Setting demo id to astralis-vs-ence-m2-nuke
13:39:

13:39:40 [INFO] Go version>=1.14.0
13:39:40 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Astralis/virtus-pro-vs-astralis-m1-overpass.dem
13:39:40 [INFO] Setting demo id to virtus-pro-vs-astralis-m1-overpass
13:39:40 [INFO] Setting parse rate to 128
13:39:40 [INFO] Go version>=1.14.0
13:39:40 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Astralis/virtus-pro-vs-astralis-m2-dust2.dem
13:39:40 [INFO] Setting demo id to virtus-pro-vs-astralis-m2-dust2
13:39:40 [INFO] Setting parse rate to 128
13:39:41 [INFO] Go version>=1.14.0
13:39:41 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Astralis/virtus-pro-vs-astralis-m3-inferno.dem
13:39:41 [INFO] Setting demo id to virtus-pro-vs-astralis-m3-inferno
13:39:41 [INFO] Setting parse rate to 128
13:39:41 [INFO] Go version>=1.14.0
13:39:41 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Astralis/vitality-vs-astralis-m1-nuke.dem
13:39:41 [INFO] Setting demo id to vitality-vs-astralis-m1-nuk

# Demo Parsers: 38


13:39:54 [INFO] Wrote demo parse output to astralis-vs-bad-news-bears-m1-inferno.json
13:39:54 [INFO] Reading in JSON from astralis-vs-bad-news-bears-m1-inferno.json
13:39:54 [INFO] JSON data loaded, available in the `json` attribute to parser
13:39:54 [INFO] Successfully parsed JSON output
13:39:54 [INFO] Successfully returned JSON output
13:39:54 [INFO] Parsed kills to Pandas DataFrame
13:39:54 [INFO] Parsed rounds to Pandas DataFrame
13:39:54 [INFO] Parsed damages to Pandas DataFrame
13:39:54 [INFO] Parsed frames to Pandas DataFrame
13:39:54 [INFO] Parsed player frames to Pandas DataFrame
13:39:54 [INFO] Parsed bomb_events to Pandas DataFrame
13:39:54 [INFO] Parsed grenades to Pandas DataFrame
13:39:54 [INFO] Parsed flashes to Pandas DataFrame
13:39:54 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:39:54 [INFO] Looking for file at /mnt/d/TOP_10/Astralis/astralis-vs-bad-news-bears-m2-nuke.dem
13:40:16 [INFO] Wrote demo parse output to astralis-vs-bad-new

13:42:33 [INFO] Looking for file at /mnt/d/TOP_10/Astralis/astralis-vs-faze-m2-nuke.dem
13:43:07 [INFO] Wrote demo parse output to astralis-vs-faze-m2-nuke.json
13:43:07 [INFO] Reading in JSON from astralis-vs-faze-m2-nuke.json
13:43:07 [INFO] JSON data loaded, available in the `json` attribute to parser
13:43:07 [INFO] Successfully parsed JSON output
13:43:07 [INFO] Successfully returned JSON output
13:43:07 [INFO] Parsed kills to Pandas DataFrame
13:43:07 [INFO] Parsed rounds to Pandas DataFrame
13:43:07 [INFO] Parsed damages to Pandas DataFrame
13:43:07 [INFO] Parsed frames to Pandas DataFrame
13:43:08 [INFO] Parsed player frames to Pandas DataFrame
13:43:08 [INFO] Parsed bomb_events to Pandas DataFrame
13:43:08 [INFO] Parsed grenades to Pandas DataFrame
13:43:08 [INFO] Parsed flashes to Pandas DataFrame
13:43:08 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:43:08 [INFO] Looking for file at /mnt/d/TOP_10/Astralis/astralis-vs-funplus-phoenix-ancient.dem

13:45:41 [INFO] Looking for file at /mnt/d/TOP_10/Astralis/astralis-vs-spirit-m3-ancient.dem
13:45:53 [INFO] Wrote demo parse output to astralis-vs-spirit-m3-ancient.json
13:45:53 [INFO] Reading in JSON from astralis-vs-spirit-m3-ancient.json
13:45:54 [INFO] JSON data loaded, available in the `json` attribute to parser
13:45:54 [INFO] Successfully parsed JSON output
13:45:54 [INFO] Successfully returned JSON output
13:45:54 [INFO] Parsed kills to Pandas DataFrame
13:45:54 [INFO] Parsed rounds to Pandas DataFrame
13:45:54 [INFO] Parsed damages to Pandas DataFrame
13:45:54 [INFO] Parsed frames to Pandas DataFrame
13:45:54 [INFO] Parsed player frames to Pandas DataFrame
13:45:54 [INFO] Parsed bomb_events to Pandas DataFrame
13:45:54 [INFO] Parsed grenades to Pandas DataFrame
13:45:54 [INFO] Parsed flashes to Pandas DataFrame
13:45:54 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:45:54 [INFO] Looking for file at /mnt/d/TOP_10/Astralis/astralis-vs-vitality-m1-

13:48:37 [INFO] Looking for file at /mnt/d/TOP_10/Astralis/heroic-vs-astralis-m1-inferno.dem
13:48:51 [INFO] Wrote demo parse output to heroic-vs-astralis-m1-inferno.json
13:48:51 [INFO] Reading in JSON from heroic-vs-astralis-m1-inferno.json
13:48:52 [INFO] JSON data loaded, available in the `json` attribute to parser
13:48:52 [INFO] Successfully parsed JSON output
13:48:52 [INFO] Successfully returned JSON output
13:48:52 [INFO] Parsed kills to Pandas DataFrame
13:48:52 [INFO] Parsed rounds to Pandas DataFrame
13:48:52 [INFO] Parsed damages to Pandas DataFrame
13:48:52 [INFO] Parsed frames to Pandas DataFrame
13:48:52 [INFO] Parsed player frames to Pandas DataFrame
13:48:52 [INFO] Parsed bomb_events to Pandas DataFrame
13:48:52 [INFO] Parsed grenades to Pandas DataFrame
13:48:52 [INFO] Parsed flashes to Pandas DataFrame
13:48:52 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:48:52 [INFO] Looking for file at /mnt/d/TOP_10/Astralis/heroic-vs-astralis-m2-ve

13:51:38 [INFO] Parsed flashes to Pandas DataFrame
13:51:38 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
13:51:38 [INFO] Looking for file at /mnt/d/TOP_10/Astralis/vitality-vs-astralis-m2-vertigo.dem
13:51:53 [INFO] Wrote demo parse output to vitality-vs-astralis-m2-vertigo.json
13:51:53 [INFO] Reading in JSON from vitality-vs-astralis-m2-vertigo.json
13:51:53 [INFO] JSON data loaded, available in the `json` attribute to parser
13:51:53 [INFO] Successfully parsed JSON output
13:51:53 [INFO] Successfully returned JSON output
13:51:53 [INFO] Parsed kills to Pandas DataFrame
13:51:53 [INFO] Parsed rounds to Pandas DataFrame
13:51:53 [INFO] Parsed damages to Pandas DataFrame
13:51:53 [INFO] Parsed frames to Pandas DataFrame
13:51:54 [INFO] Parsed player frames to Pandas DataFrame
13:51:54 [INFO] Parsed bomb_events to Pandas DataFrame
13:51:54 [INFO] Parsed grenades to Pandas DataFrame
13:51:54 [INFO] Parsed flashes to Pandas DataFrame
13:51:54 [INFO] Running Go

In [76]:
finalaKillData = concat_data(aKillData)
finalaRoundData = concat_data(aRoundData)
finalaDamageData = concat_data(aDamageData)
finalaGrenadeData = concat_data(aGrenadeData)
finalaFlashData = concat_data(aFlashData)
finalaBombData = concat_data(aBombEvents)

In [77]:
akillMapData = divideDataByMaps(finalaKillData)
aroundMapData = divideDataByMaps(finalaRoundData)
adamageMapData = divideDataByMaps(finalaDamageData)
agrenadeMapData = divideDataByMaps(finalaGrenadeData)
aflashMapData = divideDataByMaps(finalaFlashData)
aBombMapData = divideDataByMaps(finalaBombData)

In [78]:
a_kill_stats = {}
for x in akillMapData.keys():
    a_kill_stats[x] = stats.kill_stats(akillMapData[x], aroundMapData[x])

In [79]:
a_kill_stats['de_mirage']

KeyError: 'de_mirage'

In [80]:
a_kill_stats['de_overpass']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,dupreeh,86,76,13.0,10,46,11.0,1.131579,0.774775,0.534884
1,Bubzkji,73,85,15.0,-12,33,6.0,0.858824,0.657658,0.452055
2,Magisk,69,80,18.0,-11,26,17.0,0.862500,0.621622,0.376812
3,gla1ve,64,74,12.0,-10,32,10.0,0.864865,0.576577,0.500000
4,Xyp9x,50,62,14.0,-12,19,7.0,0.806452,0.450450,0.380000
5,misutaaa,29,14,3.0,15,16,5.0,2.071429,0.261261,0.551724
6,Jame,27,22,2.0,5,6,4.0,1.227273,0.243243,0.222222
7,buster,27,22,8.0,5,15,3.0,1.227273,0.243243,0.555556
8,Ax1Le,26,13,2.0,13,15,5.0,2.000000,0.234234,0.576923
9,YEKINDAR,25,26,5.0,-1,15,6.0,0.961538,0.225225,0.600000


In [81]:
a_kill_stats['de_inferno']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Magisk,143,136,36.0,7,70,20.0,1.051471,0.658986,0.489510
1,dupreeh,139,133,22.0,6,68,22.0,1.045113,0.640553,0.489209
2,Bubzkji,136,139,26.0,-3,64,22.0,0.978417,0.626728,0.470588
3,gla1ve,118,147,40.0,-29,44,21.0,0.802721,0.543779,0.372881
4,Xyp9x,79,77,14.0,2,31,11.0,1.025974,0.364055,0.392405
5,Lucky,66,53,6.0,13,23,9.0,1.245283,0.304147,0.348485
6,stavn,45,33,12.0,12,13,12.0,1.363636,0.207373,0.288889
7,cadiaN,36,28,2.0,8,6,3.0,1.285714,0.165899,0.166667
8,sjuush,35,25,8.0,10,20,7.0,1.400000,0.161290,0.571429
9,refrezh,30,31,9.0,-1,17,6.0,0.967742,0.138249,0.566667


In [82]:
a_kill_stats['de_ancient']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Magisk,74,69,13.0,5,39,13.0,1.072464,0.787234,0.527027
1,dupreeh,58,59,11.0,-1,32,10.0,0.983051,0.617021,0.551724
2,gla1ve,56,74,11.0,-18,25,9.0,0.756757,0.595745,0.446429
3,Bubzkji,54,65,10.0,-11,24,6.0,0.830769,0.574468,0.444444
4,Xyp9x,30,38,9.0,-8,14,3.0,0.789474,0.319149,0.466667
5,Lucky,29,29,7.0,0,9,3.0,1.000000,0.308511,0.310345
6,zehN,28,13,6.0,15,10,2.0,2.153846,0.297872,0.357143
7,hades,27,13,2.0,14,9,3.0,2.076923,0.287234,0.333333
8,electronic,23,13,1.0,10,12,4.0,1.769231,0.244681,0.521739
9,b1t,22,13,3.0,9,16,2.0,1.692308,0.234043,0.727273


In [83]:
a_kill_stats['de_vertigo']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Magisk,81,74,20.0,7,35,13.0,1.094595,0.801980,0.432099
1,dupreeh,74,62,10.0,12,38,8.0,1.193548,0.732673,0.513514
2,Bubzkji,59,76,16.0,-17,29,6.0,0.776316,0.584158,0.491525
3,Xyp9x,48,53,15.0,-5,23,11.0,0.905660,0.475248,0.479167
4,Lucky,31,52,4.0,-21,15,5.0,0.596154,0.306931,0.483871
5,oSee,28,19,5.0,9,13,3.0,1.473684,0.277228,0.464286
6,cadiaN,22,12,2.0,10,9,6.0,1.833333,0.217822,0.409091
7,apEX,22,10,1.0,12,17,2.0,2.200000,0.217822,0.772727
8,EliGE,21,19,3.0,2,10,5.0,1.105263,0.207921,0.476190
9,refrezh,21,13,5.0,8,9,2.0,1.615385,0.207921,0.428571


In [84]:
a_kill_stats['de_nuke']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,dupreeh,188,157,24.0,31,86.0,16.0,1.197452,0.737255,0.457447
1,Magisk,180,169,37.0,11,85.0,36.0,1.065089,0.705882,0.472222
2,Bubzkji,152,170,31.0,-18,74.0,20.0,0.894118,0.596078,0.486842
3,Xyp9x,119,124,33.0,-5,52.0,10.0,0.959677,0.466667,0.436975
4,Lucky,105,89,15.0,16,41.0,19.0,1.179775,0.411765,0.390476
5,gla1ve,98,107,20.0,-9,46.0,19.0,0.915888,0.384314,0.469388
6,misutaaa,41,34,8.0,7,23.0,1.0,1.205882,0.160784,0.560976
7,ZywOo,38,36,1.0,2,15.0,9.0,1.055556,0.149020,0.394737
8,Twistzz,38,34,8.0,4,21.0,6.0,1.117647,0.149020,0.552632
9,broky,35,31,7.0,4,6.0,9.0,1.129032,0.137255,0.171429


In [85]:
a_kill_stats['de_dust2']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Magisk,167,162,31.0,5,85.0,28.0,1.030864,0.676113,0.508982
1,dupreeh,153,160,32.0,-7,78.0,20.0,0.956250,0.619433,0.509804
2,Xyp9x,148,140,26.0,8,75.0,18.0,1.057143,0.599190,0.506757
3,Bubzkji,138,171,45.0,-33,85.0,26.0,0.807018,0.558704,0.615942
4,gla1ve,109,103,27.0,6,36.0,22.0,1.058252,0.441296,0.330275
5,Lucky,52,44,9.0,8,20.0,9.0,1.181818,0.210526,0.384615
6,s1mple,41,26,6.0,15,8.0,6.0,1.576923,0.165992,0.195122
7,YEKINDAR,40,27,3.0,13,21.0,2.0,1.481481,0.161943,0.525000
8,Jame,32,22,3.0,10,5.0,3.0,1.454545,0.129555,0.156250
9,electronic,28,32,8.0,-4,15.0,6.0,0.875000,0.113360,0.535714


In [91]:
oKillData, oRoundData, oDamageData, oFrameData, oPlayerFrames, oBombEvents, oGrenadeData, oFlashData = generateData(generateDemoParsers(getDemoPaths(og_path)))

14:13:07 [INFO] Go version>=1.14.0
14:13:07 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/OG/complexity-vs-og-m1-mirage_2350346.dem
14:13:07 [INFO] Setting demo id to complexity-vs-og-m1-mirage
14:13:07 [INFO] Setting parse rate to 128
14:13:08 [INFO] Go version>=1.14.0
14:13:08 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/OG/complexity-vs-og-m2-nuke_2350346.dem
14:13:08 [INFO] Setting demo id to complexity-vs-og-m2-nuke
14:13:08 [INFO] Setting parse rate to 128
14:13:08 [INFO] Go version>=1.14.0
14:13:08 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/OG/g2-vs-og-m1-mirage_2350336.dem
14:13:08 [INFO] Setting demo id to g2-vs-og-m1-mirage
14:13:08 [INFO] Setting parse rate to 128
14:13:09 [INFO] Go version>=1.14.0
14:13:09 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/OG/g2-vs-og-m2-inferno_2350336.dem
14:13:09 [INFO] Setting demo id to g2-vs-og-m2-inferno
14:13:09 [INFO] Setting parse rate to 128
14:13:09 [INFO] Go version

# Demo Parsers: 23


14:13:44 [INFO] Wrote demo parse output to complexity-vs-og-m1-mirage.json
14:13:44 [INFO] Reading in JSON from complexity-vs-og-m1-mirage.json
14:13:45 [INFO] JSON data loaded, available in the `json` attribute to parser
14:13:45 [INFO] Successfully parsed JSON output
14:13:45 [INFO] Successfully returned JSON output
14:13:45 [INFO] Parsed kills to Pandas DataFrame
14:13:45 [INFO] Parsed rounds to Pandas DataFrame
14:13:45 [INFO] Parsed damages to Pandas DataFrame
14:13:45 [INFO] Parsed frames to Pandas DataFrame
14:13:46 [INFO] Parsed player frames to Pandas DataFrame
14:13:46 [INFO] Parsed bomb_events to Pandas DataFrame
14:13:46 [INFO] Parsed grenades to Pandas DataFrame
14:13:46 [INFO] Parsed flashes to Pandas DataFrame
14:13:46 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:13:46 [INFO] Looking for file at /mnt/d/TOP_10/OG/complexity-vs-og-m2-nuke_2350346.dem
14:14:41 [INFO] Wrote demo parse output to complexity-vs-og-m2-nuke.json
14:14:41 [INFO] Rea

14:17:03 [INFO] JSON data loaded, available in the `json` attribute to parser
14:17:03 [INFO] Successfully parsed JSON output
14:17:03 [INFO] Successfully returned JSON output
14:17:03 [INFO] Parsed kills to Pandas DataFrame
14:17:03 [INFO] Parsed rounds to Pandas DataFrame
14:17:03 [INFO] Parsed damages to Pandas DataFrame
14:17:03 [INFO] Parsed frames to Pandas DataFrame
14:17:03 [INFO] Parsed player frames to Pandas DataFrame
14:17:03 [INFO] Parsed bomb_events to Pandas DataFrame
14:17:03 [INFO] Parsed grenades to Pandas DataFrame
14:17:03 [INFO] Parsed flashes to Pandas DataFrame
14:17:03 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:17:03 [INFO] Looking for file at /mnt/d/TOP_10/OG/og-vs-forze-m1-mirage_2350340.dem
14:17:17 [INFO] Wrote demo parse output to og-vs-forze-m1-mirage.json
14:17:17 [INFO] Reading in JSON from og-vs-forze-m1-mirage.json
14:17:17 [INFO] JSON data loaded, available in the `json` attribute to parser
14:17:17 [INFO] Successfull

14:20:26 [INFO] Parsed kills to Pandas DataFrame
14:20:26 [INFO] Parsed rounds to Pandas DataFrame
14:20:26 [INFO] Parsed damages to Pandas DataFrame
14:20:26 [INFO] Parsed frames to Pandas DataFrame
14:20:26 [INFO] Parsed player frames to Pandas DataFrame
14:20:26 [INFO] Parsed bomb_events to Pandas DataFrame
14:20:26 [INFO] Parsed grenades to Pandas DataFrame
14:20:26 [INFO] Parsed flashes to Pandas DataFrame
14:20:26 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:20:26 [INFO] Looking for file at /mnt/d/TOP_10/OG/og-vs-vitality-m1-mirage_2350386.dem
14:20:40 [INFO] Wrote demo parse output to og-vs-vitality-m1-mirage.json
14:20:40 [INFO] Reading in JSON from og-vs-vitality-m1-mirage.json
14:20:41 [INFO] JSON data loaded, available in the `json` attribute to parser
14:20:41 [INFO] Successfully parsed JSON output
14:20:41 [INFO] Successfully returned JSON output
14:20:41 [INFO] Parsed kills to Pandas DataFrame
14:20:41 [INFO] Parsed rounds to Pandas DataFra

In [92]:
finaloKillData = concat_data(oKillData)
finaloRoundData = concat_data(oRoundData)
finaloDamageData = concat_data(oDamageData)
finaloGrenadeData = concat_data(oGrenadeData)
finaloFlashData = concat_data(oFlashData)
finaloBombData = concat_data(oBombEvents)

In [93]:
okillMapData = divideDataByMaps(finaloKillData)
oroundMapData = divideDataByMaps(finaloRoundData)
odamageMapData = divideDataByMaps(finaloDamageData)
ogrenadeMapData = divideDataByMaps(finaloGrenadeData)
oflashMapData = divideDataByMaps(finaloFlashData)
oBombMapData = divideDataByMaps(finaloBombData)

In [94]:
o_kill_stats = {}
for x in okillMapData.keys():
    o_kill_stats[x] = stats.kill_stats(okillMapData[x], oroundMapData[x])

In [95]:
o_kill_stats['de_mirage']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,flameZ,140,129,26.0,11,78,33.0,1.085271,0.676329,0.557143
1,valde,138,123,19.0,15,78,15.0,1.121951,0.666667,0.565217
2,mantuu,121,118,12.0,3,34,19.0,1.025424,0.584541,0.280992
3,Aleksib,118,129,21.0,-11,47,24.0,0.914729,0.570048,0.398305
4,niko,93,106,18.0,-13,52,20.0,0.877358,0.449275,0.559140
5,niko,33,32,3.0,1,18,8.0,1.031250,0.159420,0.545455
6,NiKo,29,14,6.0,15,15,2.0,2.071429,0.140097,0.517241
7,FL1T,25,10,2.0,15,12,4.0,2.500000,0.120773,0.480000
8,broky,23,13,3.0,10,11,1.0,1.769231,0.111111,0.478261
9,blameF,22,17,4.0,5,9,3.0,1.294118,0.106280,0.409091


In [96]:
o_kill_stats['de_overpass']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Aleksib,54,51,9.0,3,28,7.0,1.058824,0.692308,0.518519
1,mantuu,46,46,8.0,0,18,2.0,1.000000,0.589744,0.391304
2,flameZ,46,58,9.0,-12,21,14.0,0.793103,0.589744,0.456522
3,valde,44,60,12.0,-16,27,6.0,0.733333,0.564103,0.613636
4,REZ,41,22,4.0,19,26,6.0,1.863636,0.525641,0.634146
5,niko,31,43,7.0,-12,17,4.0,0.720930,0.397436,0.548387
6,Plopski,23,20,6.0,3,11,2.0,1.150000,0.294872,0.478261
7,b1t,23,7,1.0,16,12,3.0,3.285714,0.294872,0.521739
8,zorte,22,16,0.0,6,7,5.0,1.375000,0.282051,0.318182
9,device,21,20,3.0,1,6,3.0,1.050000,0.269231,0.285714


In [97]:
o_kill_stats['de_inferno']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,mantuu,93,58,10.0,35,28,10.0,1.603448,0.830357,0.301075
1,flameZ,87,69,17.0,18,40,25.0,1.260870,0.776786,0.459770
2,valde,80,69,14.0,11,40,16.0,1.159420,0.714286,0.500000
3,Aleksib,56,67,22.0,-11,22,6.0,0.835821,0.500000,0.392857
4,niko,47,53,12.0,-6,25,10.0,0.886792,0.419643,0.531915
5,KENSI,33,23,4.0,10,17,2.0,1.434783,0.294643,0.515152
6,Jerry,28,25,7.0,3,9,4.0,1.120000,0.250000,0.321429
7,zorte,25,17,0.0,8,9,3.0,1.470588,0.223214,0.360000
8,rain,24,20,6.0,4,15,4.0,1.200000,0.214286,0.625000
9,NEOFRAG,22,16,2.0,6,10,4.0,1.375000,0.196429,0.454545


In [98]:
o_kill_stats['de_ancient']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Twistzz,26,18,2.0,8,18,2.0,1.444444,0.866667,0.692308
1,mantuu,25,13,0.0,12,4,7.0,1.923077,0.833333,0.160000
2,flameZ,24,22,1.0,2,15,4.0,1.090909,0.800000,0.625000
3,karrigan,21,20,3.0,1,10,5.0,1.050000,0.700000,0.476190
4,broky,20,17,1.0,3,4,3.0,1.176471,0.666667,0.200000
5,valde,17,16,6.0,1,8,1.0,1.062500,0.566667,0.470588
6,niko,15,23,1.0,-8,4,3.0,0.652174,0.500000,0.266667
7,rain,14,19,6.0,-5,7,2.0,0.736842,0.466667,0.500000
8,olofmeister,11,15,1.0,-4,6,3.0,0.733333,0.366667,0.545455
9,Aleksib,8,18,5.0,-10,2,0.0,0.444444,0.266667,0.250000


In [99]:
o_kill_stats['de_vertigo']

KeyError: 'de_vertigo'

In [100]:
o_kill_stats['de_nuke']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,mantuu,75,53,7,22,19,13.0,1.415094,0.806452,0.253333
1,flameZ,67,63,14,4,28,11.0,1.063492,0.720430,0.417910
2,niko,55,66,9,-11,36,5.0,0.833333,0.591398,0.654545
3,valde,54,65,14,-11,29,7.0,0.830769,0.580645,0.537037
4,Aleksib,50,67,14,-17,22,8.0,0.746269,0.537634,0.440000
5,NEOFRAG,36,25,4,11,16,3.0,1.440000,0.387097,0.444444
6,oskar,33,21,5,12,5,9.0,1.571429,0.354839,0.151515
7,SHOCK,27,29,4,-2,15,3.0,0.931034,0.290323,0.555556
8,ZEDKO,26,27,5,-1,14,5.0,0.962963,0.279570,0.538462
9,ZywOo,25,10,5,15,10,4.0,2.500000,0.268817,0.400000


In [101]:
o_kill_stats['de_dust2']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,valde,60,34,6.0,26,30,2.0,1.764706,0.895522,0.500000
1,flameZ,49,30,10.0,19,19,7.0,1.633333,0.731343,0.387755
2,niko,42,19,8.0,23,23,7.0,2.210526,0.626866,0.547619
3,Aleksib,40,32,13.0,8,20,8.0,1.250000,0.597015,0.500000
4,mantuu,37,28,7.0,9,5,8.0,1.321429,0.552239,0.135135
5,s1mple,20,19,2.0,1,7,6.0,1.052632,0.298507,0.350000
6,electronic,17,17,5.0,0,13,2.0,1.000000,0.253731,0.764706
7,niko,16,12,3.0,4,9,3.0,1.333333,0.238806,0.562500
8,hampus,15,17,3.0,-2,8,5.0,0.882353,0.223881,0.533333
9,Perfecto,13,15,1.0,-2,6,1.0,0.866667,0.194030,0.461538


In [137]:
bKillData, bRoundData, bDamageData, bFrameData, bPlayerFrames, bBombEvents, bGrenadeData, bFlashData = generateData(generateDemoParsers(getDemoPaths(big_path)))

14:52:47 [INFO] Go version>=1.14.0
14:52:47 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Big/big-vs-evil-geniuses-m1-nuke_2350348.dem
14:52:47 [INFO] Setting demo id to big-vs-evil-geniuses-m1-nuke
14:52:47 [INFO] Setting parse rate to 128
14:52:47 [INFO] Go version>=1.14.0
14:52:47 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Big/big-vs-evil-geniuses-m2-dust2_2350348.dem
14:52:47 [INFO] Setting demo id to big-vs-evil-geniuses-m2-dust2
14:52:47 [INFO] Setting parse rate to 128
14:52:47 [INFO] Go version>=1.14.0
14:52:48 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Big/big-vs-faze-m1-dust2_2350359.dem
14:52:48 [INFO] Setting demo id to big-vs-faze-m1-dust2
14:52:48 [INFO] Setting parse rate to 128
14:52:48 [INFO] Go version>=1.14.0
14:52:48 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Big/big-vs-faze-m2-inferno_2350359.dem
14:52:48 [INFO] Setting demo id to big-vs-faze-m2-inferno
14:52:48 [INFO] Setting parse rate to 12

# Demo Parsers: 22


14:53:28 [INFO] Wrote demo parse output to big-vs-evil-geniuses-m1-nuke.json
14:53:28 [INFO] Reading in JSON from big-vs-evil-geniuses-m1-nuke.json
14:53:29 [INFO] JSON data loaded, available in the `json` attribute to parser
14:53:29 [INFO] Successfully parsed JSON output
14:53:29 [INFO] Successfully returned JSON output
14:53:29 [INFO] Parsed kills to Pandas DataFrame
14:53:29 [INFO] Parsed rounds to Pandas DataFrame
14:53:29 [INFO] Parsed damages to Pandas DataFrame
14:53:29 [INFO] Parsed frames to Pandas DataFrame
14:53:29 [INFO] Parsed player frames to Pandas DataFrame
14:53:29 [INFO] Parsed bomb_events to Pandas DataFrame
14:53:29 [INFO] Parsed grenades to Pandas DataFrame
14:53:29 [INFO] Parsed flashes to Pandas DataFrame
14:53:29 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:53:29 [INFO] Looking for file at /mnt/d/TOP_10/Big/big-vs-evil-geniuses-m2-dust2_2350348.dem
14:53:43 [INFO] Wrote demo parse output to big-vs-evil-geniuses-m2-dust2.json
14:5

14:56:23 [INFO] Reading in JSON from big-vs-mousesports-m3-mirage.json
14:56:24 [INFO] JSON data loaded, available in the `json` attribute to parser
14:56:24 [INFO] Successfully parsed JSON output
14:56:24 [INFO] Successfully returned JSON output
14:56:24 [INFO] Parsed kills to Pandas DataFrame
14:56:24 [INFO] Parsed rounds to Pandas DataFrame
14:56:24 [INFO] Parsed damages to Pandas DataFrame
14:56:24 [INFO] Parsed frames to Pandas DataFrame
14:56:24 [INFO] Parsed player frames to Pandas DataFrame
14:56:24 [INFO] Parsed bomb_events to Pandas DataFrame
14:56:24 [INFO] Parsed grenades to Pandas DataFrame
14:56:24 [INFO] Parsed flashes to Pandas DataFrame
14:56:24 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:56:24 [INFO] Looking for file at /mnt/d/TOP_10/Big/big-vs-skade-m1-dust2_2350916.dem
14:56:49 [INFO] Wrote demo parse output to big-vs-skade-m1-dust2.json
14:56:49 [INFO] Reading in JSON from big-vs-skade-m1-dust2.json
14:56:51 [INFO] JSON data loaded,

14:59:18 [INFO] Successfully parsed JSON output
14:59:18 [INFO] Successfully returned JSON output
14:59:18 [INFO] Parsed kills to Pandas DataFrame
14:59:18 [INFO] Parsed rounds to Pandas DataFrame
14:59:18 [INFO] Parsed damages to Pandas DataFrame
14:59:18 [INFO] Parsed frames to Pandas DataFrame
14:59:18 [INFO] Parsed player frames to Pandas DataFrame
14:59:18 [INFO] Parsed bomb_events to Pandas DataFrame
14:59:18 [INFO] Parsed grenades to Pandas DataFrame
14:59:18 [INFO] Parsed flashes to Pandas DataFrame
14:59:18 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:59:18 [INFO] Looking for file at /mnt/d/TOP_10/Big/nip-vs-big-m2-inferno.dem
14:59:31 [INFO] Wrote demo parse output to nip-vs-big-m2-inferno.json
14:59:31 [INFO] Reading in JSON from nip-vs-big-m2-inferno.json
14:59:32 [INFO] JSON data loaded, available in the `json` attribute to parser
14:59:32 [INFO] Successfully parsed JSON output
14:59:32 [INFO] Successfully returned JSON output
14:59:32 [INFO

In [138]:
finalbKillData = concat_data(bKillData)
finalbRoundData = concat_data(bRoundData)
finalbDamageData = concat_data(bDamageData)
finalbGrenadeData = concat_data(bGrenadeData)
finalbFlashData = concat_data(bFlashData)
finalbBombData = concat_data(bBombEvents)

In [139]:
bkillMapData = divideDataByMaps(finalbKillData)
broundMapData = divideDataByMaps(finalbRoundData)
bdamageMapData = divideDataByMaps(finalbDamageData)
bgrenadeMapData = divideDataByMaps(finalbGrenadeData)
bflashMapData = divideDataByMaps(finalbFlashData)
bBombMapData = divideDataByMaps(finalbBombData)

In [140]:
b_kill_stats = {}
for x in bkillMapData.keys():
    b_kill_stats[x] = stats.kill_stats(bkillMapData[x], broundMapData[x])

In [141]:
b_kill_stats['de_mirage']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,syrsoN,87,64,5.0,23,19,21,1.359375,0.87,0.218391
1,tabseN,84,69,20.0,15,52,13,1.217391,0.84,0.619048
2,k1to,74,71,10.0,3,39,10,1.042254,0.74,0.527027
3,tiziaN,60,62,21.0,-2,25,3,0.967742,0.60,0.416667
4,gade,56,63,10.0,-7,27,7,0.888889,0.56,0.482143
5,device,29,15,3.0,14,10,3,1.933333,0.29,0.344828
6,frozen,28,16,9.0,12,15,3,1.750000,0.28,0.535714
7,LNZ,23,20,3.0,3,17,2,1.150000,0.23,0.739130
8,acoR,23,15,4.0,8,8,5,1.533333,0.23,0.347826
9,hampus,22,20,9.0,2,9,3,1.100000,0.22,0.409091


In [142]:
b_kill_stats['de_overpass']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Brollan,21,13,2,8,12,6.0,1.615385,0.807692,0.571429
1,Jackinho,19,12,1,7,5,3.0,1.583333,0.730769,0.263158
2,k1to,19,16,1,3,12,0.0,1.187500,0.730769,0.631579
3,syrsoN,19,16,1,3,5,4.0,1.187500,0.730769,0.263158
4,KRIMZ,15,16,3,-1,4,0.0,0.937500,0.576923,0.266667
5,gade,15,15,3,0,9,2.0,1.000000,0.576923,0.600000
6,mezii,14,12,4,2,7,3.0,1.166667,0.538462,0.500000
7,tabseN,13,18,2,-5,8,3.0,0.722222,0.500000,0.615385
8,ALEX,12,20,8,-8,4,4.0,0.600000,0.461538,0.333333
9,tiziaN,7,16,6,-9,4,1.0,0.437500,0.269231,0.571429


In [143]:
b_kill_stats['de_inferno']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,syrsoN,84,93,4,-9,28,19,0.903226,0.636364,0.333333
1,tiziaN,73,95,17,-22,50,13,0.768421,0.553030,0.684932
2,k1to,72,94,17,-22,45,9,0.765957,0.545455,0.625000
3,gade,71,91,16,-20,30,9,0.780220,0.537879,0.422535
4,tabseN,65,98,23,-33,34,6,0.663265,0.492424,0.523077
5,REZ,48,27,12,21,29,6,1.777778,0.363636,0.604167
6,device,32,20,3,12,13,5,1.600000,0.242424,0.406250
7,olofmeister,30,11,2,19,15,3,2.727273,0.227273,0.500000
8,LNZ,30,17,6,13,15,2,1.764706,0.227273,0.500000
9,Plopski,30,16,4,14,12,4,1.875000,0.227273,0.400000


In [144]:
b_kill_stats['de_ancient']

KeyError: 'de_ancient'

In [ ]:
b_kill_stats['de_vertigo']

In [ ]:
b_kill_stats['de_nuke']

In [ ]:
b_kill_stats['de_dust2']

In [102]:
sKillData, sRoundData, sDamageData, sFrameData, sPlayerFrames, sBombEvents, sGrenadeData, sFlashData = generateData(generateDemoParsers(getDemoPaths(spirit_path)))

14:29:48 [INFO] Go version>=1.14.0
14:29:48 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Spirit/spirit-vs-ago-m1-ancient.dem
14:29:48 [INFO] Setting demo id to spirit-vs-ago-m1-ancient
14:29:48 [INFO] Setting parse rate to 128
14:29:49 [INFO] Go version>=1.14.0
14:29:50 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Spirit/spirit-vs-ago-m2-nuke.dem
14:29:50 [INFO] Setting demo id to spirit-vs-ago-m2-nuke
14:29:50 [INFO] Setting parse rate to 128
14:29:50 [INFO] Go version>=1.14.0
14:29:50 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Spirit/spirit-vs-ago-m3-dust2.dem
14:29:50 [INFO] Setting demo id to spirit-vs-ago-m3-dust2
14:29:50 [INFO] Setting parse rate to 128
14:29:51 [INFO] Go version>=1.14.0
14:29:51 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Spirit/spirit-vs-skade-m1-ancient.dem
14:29:51 [INFO] Setting demo id to spirit-vs-skade-m1-ancient
14:29:51 [INFO] Setting parse rate to 128
14:29:51 [INFO] Go version>=1.

# Demo Parsers: 6


14:30:20 [INFO] Wrote demo parse output to spirit-vs-ago-m1-ancient.json
14:30:20 [INFO] Reading in JSON from spirit-vs-ago-m1-ancient.json
14:30:20 [INFO] JSON data loaded, available in the `json` attribute to parser
14:30:20 [INFO] Successfully parsed JSON output
14:30:20 [INFO] Successfully returned JSON output
14:30:20 [INFO] Parsed kills to Pandas DataFrame
14:30:20 [INFO] Parsed rounds to Pandas DataFrame
14:30:20 [INFO] Parsed damages to Pandas DataFrame
14:30:20 [INFO] Parsed frames to Pandas DataFrame
14:30:20 [INFO] Parsed player frames to Pandas DataFrame
14:30:20 [INFO] Parsed bomb_events to Pandas DataFrame
14:30:20 [INFO] Parsed grenades to Pandas DataFrame
14:30:20 [INFO] Parsed flashes to Pandas DataFrame
14:30:20 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:30:20 [INFO] Looking for file at /mnt/d/TOP_10/Spirit/spirit-vs-ago-m2-nuke.dem
14:30:41 [INFO] Wrote demo parse output to spirit-vs-ago-m2-nuke.json
14:30:41 [INFO] Reading in JSON f

In [103]:
finalsKillData = concat_data(sKillData)
finalsRoundData = concat_data(sRoundData)
finalsDamageData = concat_data(sDamageData)
finalsGrenadeData = concat_data(sGrenadeData)
finalsFlashData = concat_data(sFlashData)
finalsBombData = concat_data(sBombEvents)

In [104]:
skillMapData = divideDataByMaps(finalsKillData)
sroundMapData = divideDataByMaps(finalsRoundData)
sdamageMapData = divideDataByMaps(finalsDamageData)
sgrenadeMapData = divideDataByMaps(finalsGrenadeData)
sflashMapData = divideDataByMaps(finalsFlashData)
sBombMapData = divideDataByMaps(finalsBombData)

In [106]:
s_kill_stats = {}
for x in skillMapData.keys():
    s_kill_stats[x] = stats.kill_stats(skillMapData[x], sroundMapData[x])

In [107]:
s_kill_stats['de_mirage']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Dennyslaw,25,9,1,16,5,3.0,2.777778,1.086957,0.200000
1,Rainwaker,19,12,3,7,9,2.0,1.583333,0.826087,0.473684
2,degster,17,16,1,1,7,0.0,1.062500,0.739130,0.411765
3,sdy,17,16,3,1,9,1.0,1.062500,0.739130,0.529412
4,Duplicate,16,13,3,3,10,2.0,1.230769,0.695652,0.625000
5,KalubeR,16,15,3,1,8,2.0,1.066667,0.695652,0.500000
6,mir,15,18,2,-3,7,4.0,0.833333,0.652174,0.466667
7,magixx,14,18,4,-4,8,7.0,0.777778,0.608696,0.571429
8,SHiPZ,10,17,6,-7,7,0.0,0.588235,0.434783,0.700000
9,chopper,3,19,3,-16,2,2.0,0.157895,0.130435,0.666667


In [108]:
s_kill_stats['de_overpass']

KeyError: 'de_overpass'

In [109]:
s_kill_stats['de_inferno']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Rainwaker,24,14,3.0,10,9,1,1.714286,0.96,0.375000
1,KalubeR,19,14,1.0,5,13,1,1.357143,0.76,0.684211
2,degster,17,16,6.0,1,7,4,1.062500,0.68,0.411765
3,chopper,14,16,1.0,-2,9,4,0.875000,0.56,0.642857
4,magixx,14,18,4.0,-4,8,3,0.777778,0.56,0.571429
5,mir,14,18,1.0,-4,7,3,0.777778,0.56,0.500000
6,Dennyslaw,13,13,0.0,0,1,1,1.000000,0.52,0.076923
7,Duplicate,13,12,3.0,1,7,3,1.083333,0.52,0.538462
8,SHiPZ,13,18,5.0,-5,7,3,0.722222,0.52,0.538462
9,sdy,12,17,2.0,-5,4,2,0.705882,0.48,0.333333


In [110]:
s_kill_stats['de_ancient']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,magixx,53,43,13,10,20,11.0,1.232558,0.854839,0.377358
1,degster,50,43,5,7,22,8.0,1.162791,0.806452,0.440000
2,mir,41,41,14,0,24,6.0,1.000000,0.661290,0.585366
3,sdy,41,40,5,1,19,5.0,1.025000,0.661290,0.463415
4,chopper,37,43,10,-6,20,4.0,0.860465,0.596774,0.540541
5,Duplicate,30,29,6,1,20,6.0,1.034483,0.483871,0.666667
6,leman,23,17,1,6,14,4.0,1.352941,0.370968,0.608696
7,Rainwaker,22,26,1,-4,12,1.0,0.846154,0.354839,0.545455
8,SHiPZ,22,28,4,-6,12,4.0,0.785714,0.354839,0.545455
9,F1KU,20,16,4,4,10,1.0,1.250000,0.322581,0.500000


In [111]:
s_kill_stats['de_vertigo']

KeyError: 'de_vertigo'

In [112]:
s_kill_stats['de_nuke']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,degster,29,16,5,13,14,2.0,1.812500,1.115385,0.482759
1,leman,23,16,3,7,9,2.0,1.437500,0.884615,0.391304
2,sdy,23,17,3,6,9,3.0,1.352941,0.884615,0.391304
3,F1KU,20,22,2,-2,9,3.0,0.909091,0.769231,0.450000
4,magixx,17,16,7,1,9,2.0,1.062500,0.653846,0.529412
5,mir,16,20,7,-4,10,5.0,0.800000,0.615385,0.625000
6,rallen,16,21,2,-5,8,6.0,0.761905,0.615385,0.500000
7,Furlan,13,23,1,-10,7,2.0,0.565217,0.500000,0.538462
8,chopper,12,15,1,-3,7,1.0,0.800000,0.461538,0.583333
9,snatchie,12,16,1,-4,2,0.0,0.750000,0.461538,0.166667


In [113]:
s_kill_stats['de_dust2']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,magixx,28,16,3,12,13,3.0,1.750000,1.000000,0.464286
1,mir,28,19,5,9,15,1.0,1.473684,1.000000,0.535714
2,degster,24,16,2,8,10,3.0,1.500000,0.857143,0.416667
3,rallen,22,18,3,4,11,2.0,1.222222,0.785714,0.500000
4,snatchie,20,22,1,-2,6,6.0,0.909091,0.714286,0.300000
5,F1KU,19,23,1,-4,4,4.0,0.826087,0.678571,0.210526
6,Furlan,19,21,1,-2,8,4.0,0.904762,0.678571,0.421053
7,sdy,16,18,6,-2,8,2.0,0.888889,0.571429,0.500000
8,chopper,10,16,7,-6,9,3.0,0.625000,0.357143,0.900000
9,leman,5,22,5,-17,2,0.0,0.227273,0.178571,0.400000


In [114]:
foKillData, foRoundData, foDamageData, foFrameData, foPlayerFrames, foBombEvents, foGrenadeData, foFlashData = generateData(generateDemoParsers(getDemoPaths(forze_path)))

14:36:34 [INFO] Go version>=1.14.0
14:36:34 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Forze/blink-vs-forze-m1-overpass.dem
14:36:34 [INFO] Setting demo id to blink-vs-forze-m1-overpass
14:36:34 [INFO] Setting parse rate to 128
14:36:34 [INFO] Go version>=1.14.0
14:36:34 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Forze/blink-vs-forze-m2-inferno.dem
14:36:34 [INFO] Setting demo id to blink-vs-forze-m2-inferno
14:36:34 [INFO] Setting parse rate to 128
14:36:34 [INFO] Go version>=1.14.0
14:36:34 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Forze/forze-vs-1shot-m1-nuke.dem
14:36:34 [INFO] Setting demo id to forze-vs-1shot-m1-nuke
14:36:34 [INFO] Setting parse rate to 128
14:36:35 [INFO] Go version>=1.14.0
14:36:35 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Forze/forze-vs-1shot-m2-mirage.dem
14:36:35 [INFO] Setting demo id to forze-vs-1shot-m2-mirage
14:36:35 [INFO] Setting parse rate to 128
14:36:35 [INFO] Go version

# Demo Parsers: 8


14:37:04 [INFO] Wrote demo parse output to blink-vs-forze-m1-overpass.json
14:37:04 [INFO] Reading in JSON from blink-vs-forze-m1-overpass.json
14:37:04 [INFO] JSON data loaded, available in the `json` attribute to parser
14:37:04 [INFO] Successfully parsed JSON output
14:37:04 [INFO] Successfully returned JSON output
14:37:04 [INFO] Parsed kills to Pandas DataFrame
14:37:04 [INFO] Parsed rounds to Pandas DataFrame
14:37:04 [INFO] Parsed damages to Pandas DataFrame
14:37:04 [INFO] Parsed frames to Pandas DataFrame
14:37:04 [INFO] Parsed player frames to Pandas DataFrame
14:37:04 [INFO] Parsed bomb_events to Pandas DataFrame
14:37:04 [INFO] Parsed grenades to Pandas DataFrame
14:37:04 [INFO] Parsed flashes to Pandas DataFrame
14:37:04 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:37:04 [INFO] Looking for file at /mnt/d/TOP_10/Forze/blink-vs-forze-m2-inferno.dem
14:37:20 [INFO] Wrote demo parse output to blink-vs-forze-m2-inferno.json
14:37:20 [INFO] Readin

In [115]:
finalfoKillData = concat_data(foKillData)
finalfoRoundData = concat_data(foRoundData)
finalfoDamageData = concat_data(foDamageData)
finalfoGrenadeData = concat_data(foGrenadeData)
finalfoFlashData = concat_data(foFlashData)
finalfoBombData = concat_data(foBombEvents)

In [116]:
fokillMapData = divideDataByMaps(finalfoKillData)
foroundMapData = divideDataByMaps(finalfoRoundData)
fodamageMapData = divideDataByMaps(finalfoDamageData)
fogrenadeMapData = divideDataByMaps(finalfoGrenadeData)
foflashMapData = divideDataByMaps(finalfoFlashData)
foBombMapData = divideDataByMaps(finalfoBombData)

In [117]:
fo_kill_stats = {}
for x in fokillMapData.keys():
    fo_kill_stats[x] = stats.kill_stats(fokillMapData[x], foroundMapData[x])

In [118]:
fo_kill_stats['de_mirage']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Jerry256,48,31,3.0,17,20,6.0,1.548387,0.923077,0.416667
1,zorte-_-,38,25,5.0,13,13,9.0,1.520000,0.730769,0.342105
2,KENSI1337,38,28,7.0,10,15,4.0,1.357143,0.730769,0.394737
3,FL1TJO,27,15,3.0,12,13,4.0,1.800000,0.519231,0.481481
4,Vinny,26,18,1.0,8,16,3.0,1.444444,0.500000,0.615385
5,almazer1,22,14,5.0,8,11,2.0,1.571429,0.423077,0.500000
6,Victor,21,20,3.0,1,9,3.0,1.050000,0.403846,0.428571
7,Gabe,18,20,3.0,-2,10,5.0,0.900000,0.346154,0.555556
8,headtr1ck,18,18,1.0,0,10,2.0,1.000000,0.346154,0.555556
9,m0NESY-,17,21,1.0,-4,9,2.0,0.809524,0.326923,0.529412


In [119]:
fo_kill_stats['de_overpass']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Hank,21,12,1,9,10,1,1.750000,0.875000,0.476190
1,Jerry256,19,16,6,3,6,2,1.187500,0.791667,0.315789
2,juanflatroo,19,17,7,2,10,2,1.117647,0.791667,0.526316
3,zorte-_-,19,14,4,5,5,1,1.357143,0.791667,0.263158
4,Eric,17,18,3,-1,11,2,0.944444,0.708333,0.647059
5,Gabe,16,13,5,3,9,3,1.230769,0.666667,0.562500
6,Vinny,16,17,2,-1,5,3,0.941176,0.666667,0.312500
7,rigoN,15,20,5,-5,5,5,0.750000,0.625000,0.333333
8,Erik,11,17,2,-6,6,2,0.647059,0.458333,0.545455
9,sinnopsyy,11,21,2,-10,6,3,0.523810,0.458333,0.545455


In [120]:
fo_kill_stats['de_inferno']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,KENSI1337,67,51,8.0,16,38,14.0,1.313725,0.8375,0.567164
1,FL1TJO,53,48,12.0,5,28,5.0,1.104167,0.6625,0.528302
2,Jerry256,53,54,21.0,-1,18,11.0,0.981481,0.6625,0.339623
3,Jim,45,40,12.0,5,29,7.0,1.125000,0.5625,0.644444
4,Kevin,42,31,7.0,11,10,7.0,1.354839,0.5250,0.238095
5,Quinn,41,41,12.0,0,21,4.0,1.000000,0.5125,0.512195
6,Zim,37,42,7.0,-5,22,4.0,0.880952,0.4625,0.594595
7,gxx-,36,37,7.0,-1,8,4.0,0.972973,0.4500,0.222222
8,Rick,34,36,6.0,-2,22,3.0,0.944444,0.4250,0.647059
9,sinnopsyy,29,43,10.0,-14,17,6.0,0.674419,0.3625,0.586207


In [121]:
fo_kill_stats['de_ancient']

KeyError: 'de_ancient'

In [122]:
fo_kill_stats['de_vertigo']

KeyError: 'de_vertigo'

In [123]:
fo_kill_stats['de_nuke']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Jerry256,58,45,8.0,13,26,9.0,1.288889,0.892308,0.448276
1,FL1TJO,51,43,7.0,8,23,5.0,1.186047,0.784615,0.450980
2,KENSI1337,40,41,17.0,-1,21,8.0,0.975610,0.615385,0.525000
3,zorte-_-,30,24,1.0,6,8,4.0,1.250000,0.461538,0.266667
4,Eric,28,21,5.0,7,11,1.0,1.333333,0.430769,0.392857
5,Katalic,27,26,0.0,1,6,6.0,1.038462,0.415385,0.222222
6,HOLMES--,24,26,5.0,-2,8,5.0,0.923077,0.369231,0.333333
7,almazer1,23,18,3.0,5,11,3.0,1.277778,0.353846,0.478261
8,-FpSSSSSSSSS,23,20,3.0,3,11,3.0,1.150000,0.353846,0.478261
9,Gabe,22,15,1.0,7,7,5.0,1.466667,0.338462,0.318182


In [124]:
fo_kill_stats['de_dust2']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,m0NESY-,34,19,4,15,14,4,1.789474,1.172414,0.411765
1,Jerry256,31,19,2,12,14,3,1.631579,1.068966,0.451613
2,KENSI1337,23,19,6,4,17,4,1.210526,0.793103,0.739130
3,auNkeRe--,21,19,2,2,14,3,1.105263,0.724138,0.666667
4,FL1TJO,18,21,2,-3,8,6,0.857143,0.620690,0.444444
5,zorte-_-,16,16,5,0,5,3,1.000000,0.551724,0.312500
6,synyx--,15,20,1,-5,11,1,0.750000,0.517241,0.733333
7,fear-10-,14,20,2,-6,8,1,0.700000,0.482759,0.571429
8,almazer1,13,18,7,-5,5,2,0.722222,0.448276,0.384615
9,headtr1ck,9,24,5,-15,8,2,0.375000,0.310345,0.888889


In [126]:
cKillData, cRoundData, cDamageData, cFrameData, cPlayerFrames, cBombEvents, cGrenadeData, cFlashData = generateData(generateDemoParsers(getDemoPaths(complexity_path)))

14:44:57 [INFO] Go version>=1.14.0
14:44:57 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Complexity/complexity-vs-faze-m1-dust2.dem
14:44:57 [INFO] Setting demo id to complexity-vs-faze-m1-dust2
14:44:57 [INFO] Setting parse rate to 128
14:44:57 [INFO] Go version>=1.14.0
14:44:57 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Complexity/complexity-vs-faze-m2-nuke.dem
14:44:57 [INFO] Setting demo id to complexity-vs-faze-m2-nuke
14:44:57 [INFO] Setting parse rate to 128
14:44:58 [INFO] Go version>=1.14.0
14:44:58 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Complexity/complexity-vs-faze-m3-ancient.dem
14:44:58 [INFO] Setting demo id to complexity-vs-faze-m3-ancient
14:44:58 [INFO] Setting parse rate to 128
14:44:58 [INFO] Go version>=1.14.0
14:44:58 [INFO] Initialized CSGODemoParser with demofile /mnt/d/TOP_10/Complexity/faze-vs-complexity-m1-nuke.dem
14:44:58 [INFO] Setting demo id to faze-vs-complexity-m1-nuke
14:44:58 [INFO] Setting pa

# Demo Parsers: 7


14:45:24 [INFO] Wrote demo parse output to complexity-vs-faze-m1-dust2.json
14:45:24 [INFO] Reading in JSON from complexity-vs-faze-m1-dust2.json
14:45:25 [INFO] JSON data loaded, available in the `json` attribute to parser
14:45:25 [INFO] Successfully parsed JSON output
14:45:25 [INFO] Successfully returned JSON output
14:45:25 [INFO] Parsed kills to Pandas DataFrame
14:45:25 [INFO] Parsed rounds to Pandas DataFrame
14:45:25 [INFO] Parsed damages to Pandas DataFrame
14:45:25 [INFO] Parsed frames to Pandas DataFrame
14:45:25 [INFO] Parsed player frames to Pandas DataFrame
14:45:25 [INFO] Parsed bomb_events to Pandas DataFrame
14:45:25 [INFO] Parsed grenades to Pandas DataFrame
14:45:25 [INFO] Parsed flashes to Pandas DataFrame
14:45:25 [INFO] Running Golang parser from /home/nk095x/cs_demos/csgo/csgo/parser/
14:45:25 [INFO] Looking for file at /mnt/d/TOP_10/Complexity/complexity-vs-faze-m2-nuke.dem
14:45:45 [INFO] Wrote demo parse output to complexity-vs-faze-m2-nuke.json
14:45:45 [INF

In [127]:
finalcKillData = concat_data(cKillData)
finalcRoundData = concat_data(cRoundData)
finalcDamageData = concat_data(cDamageData)
finalcGrenadeData = concat_data(cGrenadeData)
finalcFlashData = concat_data(cFlashData)
finalcBombData = concat_data(cBombEvents)

In [128]:
ckillMapData = divideDataByMaps(finalcKillData)
croundMapData = divideDataByMaps(finalcRoundData)
cdamageMapData = divideDataByMaps(finalcDamageData)
cgrenadeMapData = divideDataByMaps(finalcGrenadeData)
cflashMapData = divideDataByMaps(finalcFlashData)
cBombMapData = divideDataByMaps(finalcBombData)

In [129]:
c_kill_stats = {}
for x in ckillMapData.keys():
    c_kill_stats[x] = stats.kill_stats(ckillMapData[x], croundMapData[x])

In [130]:
c_kill_stats['de_mirage']

KeyError: 'de_mirage'

In [131]:
c_kill_stats['de_overpass']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,Perfecto,24,12,2,12,11,3.0,2.000000,0.923077,0.458333
1,electronic,21,13,8,8,8,5.0,1.615385,0.807692,0.380952
2,s1mple,19,13,2,6,7,2.0,1.461538,0.730769,0.368421
3,blameF,16,16,3,0,4,2.0,1.000000,0.615385,0.250000
4,poizon,16,19,1,-3,9,2.0,0.842105,0.615385,0.562500
5,Boombl4,14,14,2,0,5,0.0,1.000000,0.538462,0.357143
6,coldzera,13,17,1,-4,5,3.0,0.764706,0.500000,0.384615
7,jks,13,18,1,-5,7,2.0,0.722222,0.500000,0.538462
8,b1t,10,16,1,-6,9,4.0,0.625000,0.384615,0.900000
9,es3tag,10,18,2,-8,6,3.0,0.555556,0.384615,0.600000


In [132]:
c_kill_stats['de_inferno']

KeyError: 'de_inferno'

In [133]:
c_kill_stats['de_ancient']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,es3tag,33,23,6.0,10,12,3,1.434783,0.916667,0.363636
1,poizon,23,23,1.0,0,6,3,1.000000,0.638889,0.260870
2,coldzera,22,26,3.0,-4,14,3,0.846154,0.611111,0.636364
3,blameF,20,24,5.0,-4,7,2,0.833333,0.555556,0.350000
4,olofmeister,20,4,3.0,16,11,2,5.000000,0.555556,0.550000
5,jks,18,24,5.0,-6,9,1,0.750000,0.500000,0.500000
6,broky,17,5,2.0,12,3,2,3.400000,0.472222,0.176471
7,rain,17,7,3.0,10,5,3,2.428571,0.472222,0.294118
8,Twistzz,13,5,4.0,8,6,1,2.600000,0.361111,0.461538
9,karrigan,13,8,0.0,5,6,5,1.625000,0.361111,0.461538


In [134]:
c_kill_stats['de_vertigo']

KeyError: 'de_vertigo'

In [135]:
c_kill_stats['de_nuke']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,es3tag,60,42,7.0,18,20,11.0,1.428571,0.833333,0.333333
1,poizon,55,49,13.0,6,25,11.0,1.122449,0.763889,0.454545
2,blameF,46,33,12.0,13,16,9.0,1.393939,0.638889,0.347826
3,jks,46,40,9.0,6,22,3.0,1.150000,0.638889,0.478261
4,coldzera,42,37,8.0,5,16,5.0,1.135135,0.583333,0.380952
5,rain,31,39,6.0,-8,19,8.0,0.794872,0.430556,0.612903
6,broky,29,31,2.0,-2,6,3.0,0.935484,0.402778,0.206897
7,s1mple,23,14,0.0,9,12,2.0,1.642857,0.319444,0.521739
8,Twistzz,22,33,3.0,-11,14,2.0,0.666667,0.305556,0.636364
9,karrigan,18,36,8.0,-18,9,5.0,0.500000,0.250000,0.500000


In [136]:
c_kill_stats['de_dust2']

,Player,K,D,A,+/-,HS,FK,KDR,KPR,HS%
0,jks,26,18,3,8,22,2.0,1.444444,0.866667,0.846154
1,broky,24,16,3,8,10,5.0,1.500000,0.800000,0.416667
2,rain,21,20,7,1,14,7.0,1.050000,0.700000,0.666667
3,blameF,18,19,4,-1,4,3.0,0.947368,0.600000,0.222222
4,olofmeister,18,17,2,1,13,2.0,1.058824,0.600000,0.722222
5,Twistzz,16,19,4,-3,9,0.0,0.842105,0.533333,0.562500
6,poizon,16,19,4,-3,3,4.0,0.842105,0.533333,0.187500
7,es3tag,15,18,2,-3,5,4.0,0.833333,0.500000,0.333333
8,karrigan,15,18,4,-3,5,2.0,0.833333,0.500000,0.333333
9,coldzera,14,21,2,-7,4,1.0,0.666667,0.466667,0.285714


In [ ]:
eKillData, eRoundData, eDamageData, eFrameData, ePlayerFrames, eBombEvents, eGrenadeData, eFlashData = generateData(generateDemoParsers(getDemoPaths(ence_path)))

In [ ]:
finaleKillData = concat_data(eKillData)
finaleRoundData = concat_data(eRoundData)
finaleDamageData = concat_data(eDamageData)
finaleGrenadeData = concat_data(eGrenadeData)
finaleFlashData = concat_data(eFlashData)
finaleBombData = concat_data(eBombEvents)

In [ ]:
ekillMapData = divideDataByMaps(finaleKillData)
eroundMapData = divideDataByMaps(finaleRoundData)
edamageMapData = divideDataByMaps(finaleDamageData)
egrenadeMapData = divideDataByMaps(finaleGrenadeData)
eflashMapData = divideDataByMaps(finaleFlashData)
eBombMapData = divideDataByMaps(finaleBombData)

In [ ]:
e_kill_stats = {}
for x in ekillMapData.keys():
    e_kill_stats[x] = stats.kill_stats(ekillMapData[x], eroundMapData[x])

In [ ]:
e_kill_stats['de_mirage']

In [ ]:
e_kill_stats['de_overpass']

In [ ]:
e_kill_stats['de_inferno']

In [ ]:
e_kill_stats['de_ancient']

In [ ]:
e_kill_stats['de_vertigo']

In [ ]:
e_kill_stats['de_nuke']

In [ ]:
e_kill_stats['de_dust2']

In [ ]:
mKillData, mRoundData, mDamageData, mFrameData, mPlayerFrames, mBombEvents, mGrenadeData, mFlashData = generateData(generateDemoParsers(getDemoPaths(mous_path)))

In [ ]:
finalmKillData = concat_data(mKillData)
finalmRoundData = concat_data(mRoundData)
finalmDamageData = concat_data(mDamageData)
finalmGrenadeData = concat_data(mGrenadeData)
finalmFlashData = concat_data(mFlashData)
finalmBombData = concat_data(mBombEvents)

In [ ]:
mkillMapData = divideDataByMaps(finalmKillData)
mroundMapData = divideDataByMaps(finalmRoundData)
mdamageMapData = divideDataByMaps(finalmDamageData)
mgrenadeMapData = divideDataByMaps(finalmGrenadeData)
mflashMapData = divideDataByMaps(finalmFlashData)
mBombMapData = divideDataByMaps(finalmBombData)

In [ ]:
m_kill_stats = {}
for x in mkillMapData.keys():
    m_kill_stats[x] = stats.kill_stats(mkillMapData[x], mroundMapData[x])

In [ ]:
m_kill_stats['de_mirage']

In [ ]:
m_kill_stats['de_overpass']

In [ ]:
m_kill_stats['de_inferno']

In [ ]:
m_kill_stats['de_ancient']

In [ ]:
m_kill_stats['de_vertigo']

In [ ]:
m_kill_stats['de_nuke']

In [ ]:
m_kill_stats['de_dust2']